In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910460 sha256=34762d819e8fedd9a1be98a104d77a3b57ddc29e576aef96448e489fa8fa89a5
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [ ]:
import os


# Change the current directory to 4D-FedGNN-Plus
os.chdir("/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine")

# Print the current directory to confirm
print(os.getcwd())

/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine


In [ ]:
from dataset import *
from demo import *
import main_functions as mf


running on GPU
running on GPU


In [ ]:
import torch
import copy
import torch.nn as nn
from torch.nn.utils import weight_norm
from torch_geometric.nn import NNConv
from torch_geometric.data import Data, InMemoryDataset, DataLoader
from torch.nn import Linear, Sequential, ReLU
from data_utils import create_edge_index_attribute
from torch.nn.parameter import Parameter
from torch import mm as mm
from torch.nn import Tanh
import torch.nn.functional as F
import numpy as np
import random
from collections import defaultdict
from itertools import combinations

# Load the dataset

In [ ]:
args = get_args()

In [ ]:
import scipy.io

data_0 = scipy.io.loadmat('/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Data/data_t0.mat')
data_1 = scipy.io.loadmat('/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Data/data_t1.mat')
data_2 = scipy.io.loadmat('/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Data/data_t2.mat')

data_0 = data_0['data_t0']
data_1 = data_1['data_t1']
data_2 = data_2['data_t2']

X = np.zeros([len(data_0),3,35,35])
for idx,(sample_0,sample_1,sample_2) in enumerate(zip(data_0,data_1,data_2)):

        matrix_0 = antiVectorize(sample_0,35)
        matrix_1 = antiVectorize(sample_1,35)
        matrix_2 = antiVectorize(sample_2,35)

        X[idx,0,:,:]=matrix_0
        X[idx,1,:,:]=matrix_1
        X[idx,2,:,:]=matrix_2

In [ ]:
# Convert X to tensor
X = torch.from_numpy(X)
X = X.type(torch.FloatTensor)

In [ ]:
# Create table
table = np.zeros((args.num_folds - 1, args.num_timepoints))
table = random_table(args, 4/8)
dataset = copy.deepcopy(X)

In [ ]:
# remove element 88
dataset = np.delete(dataset,88,axis=0)
# dataset shape
print(f'Dataset shape:{dataset.shape}')

Dataset shape:torch.Size([113, 3, 35, 35])


# Check

In [ ]:
# function to check our results
def check(name='4D-FED-GNN++',seed=200,ratio='4_8'):
   total_final = []
   for h in range(4):
      total = np.array([0,0],dtype=np.float32)
      for f in range(5):
            total += np.load(f"/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_{seed}/{ratio}/global_test/GNNs/{name}/test_mae_losses/mae_test_loss_hospital_{h}_fold_{f}.npy")
      print(f'Hospital {h}:')
      print(total/5)
      total_final.append(total/5)

   total_final = np.array(total_final)
   total_final = total_final.sum(axis=0)/4
   print()
   print(f'Total sum:{total_final}')


# Set up the network architecture and train-validate

**Train-validate-functions**

In [ ]:
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import Data
import networkx as nx

In [ ]:
def node_features_from_adj_matrix(adj_matrix,device):

      if device.type=='cpu':
          # Create a NetworkX graph from the adjacency matrix
          G = nx.from_numpy_array(adj_matrix.numpy())
      elif device.type=='cuda':
          # Create a NetworkX graph from the adjacency matrix
          G = nx.from_numpy_array(adj_matrix.detach().cpu().numpy())

      # Compute the weighted degree (strength) for each node
      strength = dict(G.degree(weight='weight'))

      # Compute the degree for each node
      degree = dict(G.degree())

      # Compute the clustering coefficient for each node
      clustering = nx.clustering(G, weight='weight')

      # Compute the closeness centrality for each node
      closeness_centrality = nx.closeness_centrality(G)

      # Compute the PageRank for each node
      pagerank = nx.pagerank(G, weight='weight')

      # Let's convert these features into numpy arrays so we can stack them together
      strength_array = np.array(list(strength.values()))
      degree_array = np.array(list(degree.values()))
      clustering_array = np.array(list(clustering.values()))
      closeness_centrality_array = np.array(list(closeness_centrality.values()))
      pagerank_array = np.array(list(pagerank.values()))

      # Now we can stack these features together to get a node feature matrix
      x = torch.Tensor(np.vstack([strength_array, degree_array, clustering_array, closeness_centrality_array, pagerank_array]).T)
      return x


In [ ]:
def adj_matrix_to_pytorch_geometric_data(adj_matrix,device):

    # calculate edge_index and edge_weights
    edge_indices, edge_weights = dense_to_sparse(adj_matrix)

    #edge attributes
    edge_attr = torch.cat([edge_indices.T,edge_weights.view(len(edge_weights),1)],1)

    # calculate the node features
    x = node_features_from_adj_matrix(adj_matrix,device)

    data = Data(x=x.to(device),edge_index=edge_indices.to(device),edge_weights=edge_weights.to(device),adj_matrix=adj_matrix.to(device),edge_attr=edge_attr.to(device))

    return data


In [ ]:
def create_edge_index_attribute_new(adj_matrix):
    """
    Given an adjacency matrix, this function creates the edge index and edge attribute matrix
    suitable to graph representation in PyTorch Geometric.
    """

    rows, cols = adj_matrix.shape[0], adj_matrix.shape[1]
    edge_index = [[],[]]
    edge_attr = []

    for i in range(rows):
        for j in range(cols):

              if adj_matrix[i,j] > 0:

                    edge_index[0].append(i)
                    edge_index[1].append(j)
                    edge_attr.append(adj_matrix[i,j])

    return torch.tensor(edge_index), torch.Tensor(edge_attr), rows, cols

In [ ]:
def get_adjacency_matrix(num_nodes, edge_indices, edge_weights):
    # Initialize an empty adjacency matrix
    adjacency_matrix = np.zeros((num_nodes, num_nodes))

    # Fill in the adjacency matrix
    for ((node1, node2), weight) in zip(edge_indices, edge_weights):
        adjacency_matrix[node1, node2] = weight

    return adjacency_matrix


In [ ]:
def train_gnns_final(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1):
        """
            Arguments:
            args: arguments
            dataset: the whole dataset (train and test set)
            table: [num_hospitals, num_timepoints], holds timepoint-wise availability of hospitals

        This function performs training and testing reporting Mean Absolute Error (MAE) of the testing brain graphs.
        """

        # Create the results folders
        print('Train NEW')
        os.makedirs(args.save_path, exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/real_and_predicted_graphs', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/tp_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/total_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/test_mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/trained_models', exist_ok=True)


        # Create the results folders
        manualSeed = seed

        np.random.seed(manualSeed)
        random.seed(manualSeed)
        torch.manual_seed(manualSeed)

        # show the fixed seed
        print(f'Fixed seed:{seed}')
        print()

        # create the table that shows the data availability by  timepoint
        table = np.zeros((args.num_folds - 1, args.num_timepoints))
        table = random_table(args, ratio)
        print(f'Table:')
        print(table)
        print(f'Ratio:{ratio}')
        print()

        if torch.cuda.is_available():
            device = torch.device('cuda:0')
            print('running on GPU')
            # if you are using GPU
            torch.cuda.manual_seed(manualSeed)
            torch.cuda.manual_seed_all(manualSeed)

            torch.backends.cudnn.enabled = False
            torch.backends.cudnn.benchmark = False
            torch.backends.cudnn.deterministic = False
            os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' # !!! necessary for the below line
            torch.use_deterministic_algorithms(False)
            print('TRAIN deterministic algorithms')

        else:
            device = torch.device("cpu")
            print('running on CPU')

        # change the save path
        args.save_path = args.save_path+f'{args.save_name}/'

        # Choosing the right get_order function
        if args.mode == 'weighted_weight_exchange':
            get_order =  get_order_weighted
        else:
            get_order =  get_order_gnns


        # Getting our fold dict
        fold_dict,X = mf.create_fold_dict_new(dataset,num_hospitals=4,num_folds=5)

        # Perform the 5-fold Cross-Validation
        num_hospitals = args.num_folds - 1
        for f in range(args.num_folds):

            # fix the seeds
            np.random.seed(manualSeed)
            random.seed(manualSeed)
            torch.manual_seed(manualSeed)

            tic0 = timeit.default_timer()

            print(
                f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')

                        # Create hospitals
            hospitals = []
            for i in range(num_hospitals):
                  hospitals.append(Hospital_meta(args,device))
                  print('GAT')


            # Train data for each hospital
            train_data_list = []

            # Current test data
            test_data = X[-1][f].to(device)

            for i in range(num_hospitals):

                # Append the fold related to Hospital i - fold f. Note, here we append the real data, not the indices
                train_data_list.append(X[i][fold_dict[f'Hospital_{i}'][f'fold_{f}'][0]].to(device))


            # Start measuring the epochs time
            epochs_start = time.time()

            # Initiate Training
            for epoch in range(args.num_epochs):

                  epoch +=1

                  # order the hospitals based on the data availability
                  ordered_hospitals = get_order_gnns(table)

                  for h_i,hospital in enumerate(hospitals):

                    # get the train data for the hospital
                    train_data = train_data_list[h_i]

                    # get the table for th current hospital
                    table_hospital = table[h_i]

                    #if h_i ==3:
                     # return args,hospital,train_data,table_hospital

                    # Train the current hospital at the current timepoint for 1 epoch
                    mae_loss,hospital = train_one_epoch_gnns(args,hospital,train_data,table_hospital)

                    # Updating the hospital
                    hospitals[h_i] = hospital

                    if verbose:
                        print(f'Epoch:{epoch} Hospital:{h_i}, Train MAE Loss:{mae_loss}')


                  # Perform validation during training
                  if train_validate_verbose and (epoch%train_validate_verbosity_epochs==0 or epoch==1):

                      val_loss,val_mean_loss = validate_during_training_gnns(args, hospitals, test_data)
                      print(f"Epoch:{epoch},val_loss:")
                      print(f'Total MAE Loss')
                      print(val_loss)
                      print()
                      print(f'Average MAE Loss:')
                      print(val_mean_loss)
                      print()

                      for h_i,l in enumerate(val_mean_loss):

                          hospitals[h_i].scheduler.step(l)


                  if epoch != args.num_epochs - 1 or epoch != 0:
                      if epoch % args.C == 0 and args.mode != "4D-GNN":
                          print('Central Aggregation')
                          hospitals = update_main_by_average_gnns(hospitals)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN+":
                          hospitals = exchange_models(hospitals, t)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN++":
                          print('4D-FED-GNN++')
                          hospitals = exchange_models_based_on_order_gnns(hospitals, ordered_hospitals)
                      elif epoch % args.D == 0 and args.mode == "weighted_weight_exchange":
                          print('weighted_weight_exchange')
                          hospitals = exchange_models_weights_pairs_extreme(hospitals, t, ordered_hospitals)



            epochs_end = time.time()-epochs_start
            print()
            print(f'epochs finished with time:{epochs_end}')
            print()
            validate_gnns(args, hospitals, test_data, f)
            tic1 = timeit.default_timer()
            timer(tic0,tic1)




In [ ]:
def exchange_models_based_on_order_gnns(hospitals, order):
    """
        This function exchanges GNN-layer weights of paired hospitals at timepoint t with each other
    """
    pre_model = None
    for i, h_i in enumerate(order):
        next_model = copy.deepcopy(hospitals[h_i].model.state_dict())

        if not pre_model is None:
            hospitals[h_i].model.load_state_dict(pre_model)

        pre_model = copy.deepcopy(next_model)

        if i == 0:
            hospitals[h_i].model.load_state_dict(copy.deepcopy(hospitals[order[-1]].model.state_dict()))

    return hospitals

In [ ]:
def update_main_by_average_gnns(hospitals):
    """
        This function takes the GNN-layer weights of the GNN at timepoint t and computes the global model by averaging,
        then broadcats the weights to the hospitals (updates each GNN with the global model)
    """
    target_state_dict = copy.deepcopy(hospitals[0].model.state_dict())
    mux = 1 / len(hospitals)


    model_state_dict_list = [copy.deepcopy(hospitals[i].model.state_dict()) for i in range(1, len(hospitals))]


    for key in target_state_dict:
        if target_state_dict[key].data.dtype == torch.float32:
            target_state_dict[key].data = target_state_dict[key].data.clone() * mux
            for model_state_dict in model_state_dict_list:
                target_state_dict[key].data += mux * model_state_dict[key].data.clone()

    for i in range(len(hospitals)):
        hospitals[i].model.load_state_dict(target_state_dict)

    return hospitals

In [ ]:
def validate_during_training_gnns(args, hospitals, test_data):
    """
    This function calculates the average MAE of predicted brain graphs during validation.
    """
    mael = torch.nn.L1Loss()

    hloss=np.array([np.zeros(args.num_timepoints-1) for i in range(args.num_hospitals)])
    for h_i, hospital in enumerate(hospitals):
           hospital.model.eval()

           with torch.no_grad():


                for data in test_data:

                    input = data[0]
                    mae_loss_hospital = np.zeros(args.num_timepoints-1)

                    for t in range(args.num_timepoints-1):

                        pred = hospital.model(input)
                        hloss[h_i,t] +=  mael(pred, data[t+1])
                        input = pred


    # Calculate and save the average MAE Loss for each hospital
    avg_hloss = hloss/len(test_data)
    avg_hloss_mean = np.mean(avg_hloss,axis=1)
    return avg_hloss,avg_hloss_mean

In [ ]:
def validate_gnns(args, hospitals, test_data,f,verbose=False):
    """
    This function calculates the average MAE of predicted brain graphs during validation.
    """
    mael = torch.nn.L1Loss()

    hloss=np.array([np.zeros(args.num_timepoints-1) for i in range(args.num_hospitals)])
    for h_i, hospital in enumerate(hospitals):
           hospital.model.eval()

           with torch.no_grad():


                for subject_index,data in enumerate(test_data):

                    input = data[0]
                    mae_loss_hospital = np.zeros(args.num_timepoints-1)

                    for t in range(args.num_timepoints-1):

                        pred = hospital.model(input)
                        hloss[h_i,t] +=  mael(pred, data[t+1])
                        input = pred

                        #plot and save the brain graph of patient(sample) i through all the timepoints
                        plot_matrix(data[t+1].cpu().detach().numpy(),f'Real Graph, Hospital:{h_i}, Subject:{subject_index}, Timepoint:{t+1}',
                                  args.save_path+'real_and_predicted_graphs/'+f'hospital_{h_i}_subject_{subject_index}_timepoint_{t+1}_fold_{f}_real_graph',verbose)
                        plot_matrix(pred.cpu().detach().numpy(),f'Predicted Graph, Hospital:{h_i}, Subject:{subject_index}, Timepoint:{t+1}',
                                  args.save_path+'real_and_predicted_graphs/'+f'hospital_{h_i}_subject_{subject_index}_timepoint_{t+1}_fold_{f}_predicted_graph',verbose)



    # Calculate and save the average MAE Loss for each hospital
    avg_hloss = hloss/len(test_data)
    for h_i,loss in enumerate(avg_hloss):

        # Save the MAE Loss
        np.save(args.save_path+f"test_mae_losses/mae_test_loss_hospital_{h_i}_fold_{f}", loss)
    # Save the loss

    print(avg_hloss)
    return avg_hloss

In [ ]:
class Hospital_gnns():
    def __init__(self, args,device):
        """
        Hospital object contains a GNN and an optimizer for each timepoint

        Hospital object can update GNN-layer wise weights of its GNNs
        """

        self.model = RGCN(in_channels=3,hidden_size=32,num_nodes=35,window=1,dropout=0.4,device=device).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=args.lr)



In [ ]:
device = torch.device('cuda:0')

In [ ]:
def train_one_epoch_gnns(args,hospital,train_data,table_hospital):
      # Set the model in training mode
      hospital.model.train()

      num_timepoints = train_data.shape[1]

      #loss types definition
      MAE_loss = torch.nn.L1Loss()

      # this is our loss for all the data

      mae_loss_overall = []

      # loop through the data batches
      for data_id,data in enumerate(train_data):

            # zero the gradients
            hospital.optimizer.zero_grad()

            num_preds=0 # the number of times that we are able to predict
            mae_loss = 0
            t=0

            # loop through the time dependent adj matrices in the batches
            while t < num_timepoints-1:
                #print(data_id,t)

                # check if the next timepoint is available
                if table_hospital[t+1]==1:


                    pred = hospital.model(data[t])
                    real = data[t+1]

                    mae_loss += MAE_loss(pred,real)
                    #print(f'MAE LOSS FROM CURRENT PREDICTION:{MAE_loss(pred,real) }')
                    num_preds+=1
                    t+=1

                # if the next timepoint is not available
                elif table_hospital[t+1]==0:

                    pred = hospital.model(data[t])
                    #return pred


                    # find the next closes available 1 to use it for a label. We use the pred as input until then
                    reached = False
                    for t in range(t+1,num_timepoints-1):
                        #print(f'Search t:{t}')


                        # using the previous prediction as input
                        pred = hospital.model(pred)

                        # if the next timepoint has data available we break
                        if table_hospital[t+1]==1:

                                real = data[t+1]
                                mae_loss += MAE_loss(pred,real)
                                #print(f'MAE LOSS FROM CURRENT PREDICTION:{MAE_loss(pred,real)}')
                                num_preds+=1
                                t+=1
                                reached = True
                                break

                    # if there are no more 1s in our table, we break from the loop and use whatever losses we accumulated in this loop
                    if not reached:
                      break

                #print()
                #print(f'the new t is:{t}')
                #print()

            #print(mae_loss)
            # Calculate the total MAE Loss for the current batch

            if num_preds == 0:
              return 100,hospital

            mae_loss=mae_loss/num_preds
            # Append to the total MAE Los
            mae_loss_overall.append(mae_loss.item())
            #print(f'Num Predictions:{num_preds}')

            # Update the weights of the neural network
            mae_loss.backward()
            hospital.optimizer.step()



      mae_loss_overall = np.mean(np.array(mae_loss_overall))
      return mae_loss_overall,hospital


In [ ]:
def get_order_gnns(table):

    hospital_distances=[]
    hospital_number_of_points=[]
    for row in table:

        hospital_number_of_points.append(int(row.sum()))
        ones_indexes = np.where(row==1)[0]
        distances = np.diff(ones_indexes)-1
        distance = distances.sum()
        hospital_distances.append(distance)


    hospital_number_of_points = np.array(hospital_number_of_points)
    hospital_distances = np.array(hospital_distances)
    order = np.lexsort((-hospital_distances,hospital_number_of_points))
    order = np.flip(order)



    return order


In [ ]:
def random_table_gnns(args, ratio):
    # Creating the initial table filled with zeros
    table = np.zeros((args.num_hospitals, args.num_timepoints))

    # Filling out the first column with ones
    table[:,0] = 1

    # Randomly assigning a one to each row
    for i in range(args.num_hospitals):
        random_index = np.random.randint(1, args.num_timepoints)
        table[i, random_index] = 1

    # Getting the indices of the remaining zeros
    zero_indices = np.argwhere(table == 0)

    # Shuffling the indices
    np.random.shuffle(zero_indices)

    # Calculating how many of the zeros should be replaced with ones
    num_replace = int(len(zero_indices) * ratio)

    # Replacing the zeros with ones
    for i in range(num_replace):
        table[zero_indices[i,0], zero_indices[i,1]] = 1

    return table

## GAT

In [ ]:
from torch_geometric.nn import GATConv

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self,device):
        super(GAT, self).__init__()
        self.in_channels = 5
        self.hidden_channels = 64
        self.heads=8
        self.out_head = 1
        self.device=device
        self.num_nodes = 35


        self.conv1 = GATConv(self.in_channels, self.hidden_channels, heads=self.heads, dropout=0.6)
        self.conv2 = GATConv(self.hidden_channels*self.heads, self.hidden_channels, concat=False,
                             heads=self.out_head, dropout=0.6)
        self.fc = nn.Linear(self.hidden_channels,self.num_nodes)


    def forward(self, adj_matrix):


        data = adj_matrix_to_pytorch_geometric_data(adj_matrix,self.device)
        edge_index = data.edge_index
        #print(data.x)
        x = F.dropout(data.x, p=0.6, training=self.training)
        data = self.conv1(x, edge_index)
        data = F.elu(data)
        data = F.dropout(data, p=0.6, training=self.training)
        data = self.conv2(data, edge_index)
        out = F.relu(self.fc(data))

        return out

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self,device):
        super(GAT, self).__init__()
        self.in_channels = 35
        self.hidden_channels = 120
        self.heads=16
        self.out_head = 1
        self.device=device
        self.num_nodes = 35

        self.conv1 = GATConv(self.in_channels, self.hidden_channels, heads=self.heads, dropout=0.6)
        self.conv2 = GATConv(self.hidden_channels*self.heads, self.hidden_channels, heads=self.heads, dropout=0.6)
        self.conv3 = GATConv(self.hidden_channels*self.heads, self.hidden_channels, concat=False,
                             heads=self.out_head, dropout=0.6)
        self.fc = nn.Linear(self.hidden_channels,self.num_nodes)

    def forward(self, adj_matrix):

        data = adj_matrix_to_pytorch_geometric_data(adj_matrix,self.device)
        edge_index = data.edge_index

        x = F.dropout(data.adj_matrix, p=0.6, training=self.training)
        data = self.conv1(x, edge_index)
        data = F.elu(data)
        data = F.dropout(data, p=0.6, training=self.training)
        data = self.conv2(data, edge_index)
        data = F.elu(data)
        data = F.dropout(data, p=0.6, training=self.training)
        data = self.conv3(data, edge_index)
        out = F.relu(self.fc(data))

        return out


In [ ]:
torch.use_deterministic_algorithms(False)

In [ ]:
data = dataset[0][0].to(device)
gat = GAT(device).to(device)
gat(data).shape

torch.Size([35, 35])

In [ ]:
model = GAT(device)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total parameters:", total_params)
print("Trainable parameters:", trainable_params)

Total parameters: 4000115
Trainable parameters: 4000115


In [ ]:
device = torch.device('cuda:0')

In [ ]:
class Hospital_gnns():
    def __init__(self, args,device):
        """
        Hospital object contains a GNN and an optimizer for each timepoint

        Hospital object can update GNN-layer wise weights of its GNNs
        """

        self.model = GAT(device=device).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min',factor=0.5,patience=3,verbose=True)

In [ ]:
def train_gnns_final(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1):
        """
            Arguments:
            args: arguments
            dataset: the whole dataset (train and test set)
            table: [num_hospitals, num_timepoints], holds timepoint-wise availability of hospitals

        This function performs training and testing reporting Mean Absolute Error (MAE) of the testing brain graphs.
        """

        # Create the results folders
        print('Train NEW')
        os.makedirs(args.save_path, exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/real_and_predicted_graphs', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/tp_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/total_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/test_mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/trained_models', exist_ok=True)


        # Create the results folders
        manualSeed = seed

        np.random.seed(manualSeed)
        random.seed(manualSeed)
        torch.manual_seed(manualSeed)

        # show the fixed seed
        print(f'Fixed seed:{seed}')
        print()

        # create the table that shows the data availability by  timepoint
        table = np.zeros((args.num_folds - 1, args.num_timepoints))
        table = random_table(args, ratio)
        print(f'Table:')
        print(table)
        print(f'Ratio:{ratio}')
        print()

        if torch.cuda.is_available():
            device = torch.device('cuda:0')
            print('running on GPU')
            # if you are using GPU
            torch.cuda.manual_seed(manualSeed)
            torch.cuda.manual_seed_all(manualSeed)

            torch.backends.cudnn.enabled = False
            torch.backends.cudnn.benchmark = False
            torch.backends.cudnn.deterministic = False
            os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' # !!! necessary for the below line
            torch.use_deterministic_algorithms(False)
            print('TRAIN deterministic algorithms')

        else:
            device = torch.device("cpu")
            print('running on CPU')

        # change the save path
        args.save_path = args.save_path+f'{args.save_name}/'

        # Choosing the right get_order function
        if args.mode == 'weighted_weight_exchange':
            get_order =  get_order_weighted
        else:
            get_order =  get_order_gnns


        # Getting our fold dict
        fold_dict,X = mf.create_fold_dict_new(dataset,num_hospitals=4,num_folds=5)

        # Perform the 5-fold Cross-Validation
        num_hospitals = args.num_folds - 1
        for f in range(args.num_folds):

            # fix the seeds
            np.random.seed(manualSeed)
            random.seed(manualSeed)
            torch.manual_seed(manualSeed)

            tic0 = timeit.default_timer()

            print(
                f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')

                        # Create hospitals
            hospitals = []
            for i in range(num_hospitals):
                  hospitals.append(Hospital_gnns(args,device))
                  print('GAT')


            # Train data for each hospital
            train_data_list = []

            # Current test data
            test_data = X[-1][f].to(device)

            for i in range(num_hospitals):

                # Append the fold related to Hospital i - fold f. Note, here we append the real data, not the indices
                train_data_list.append(X[i][fold_dict[f'Hospital_{i}'][f'fold_{f}'][0]].to(device))


            # Start measuring the epochs time
            epochs_start = time.time()

            # Initiate Training
            for epoch in range(args.num_epochs):

                  epoch +=1

                  # order the hospitals based on the data availability
                  ordered_hospitals = get_order_gnns(table)

                  for h_i,hospital in enumerate(hospitals):

                    # get the train data for the hospital
                    train_data = train_data_list[h_i]

                    # get the table for th current hospital
                    table_hospital = table[h_i]

                    # Train the current hospital at the current timepoint for 1 epoch
                    mae_loss,hospital = train_one_epoch_gnns(args,hospital,train_data,table_hospital)

                    # Updating the hospital
                    hospitals[h_i] = hospital

                    if verbose:
                        print(f'Epoch:{epoch} Hospital:{h_i}, Train MAE Loss:{mae_loss}')


                  # Perform validation during training
                  if train_validate_verbose and (epoch%train_validate_verbosity_epochs==0 or epoch==1):

                      val_loss,val_mean_loss = validate_during_training_gnns(args, hospitals, test_data)
                      print(f"Epoch:{epoch},val_loss:")
                      print(f'Total MAE Loss')
                      print(val_loss)
                      print()
                      print(f'Average MAE Loss:')
                      print(val_mean_loss)
                      print()

                      for h_i,l in enumerate(val_mean_loss):

                          hospitals[h_i].scheduler.step(l)


                  if epoch != args.num_epochs - 1 or epoch != 0:
                      if epoch % args.C == 0 and args.mode != "4D-GNN":
                          print('Central Aggregation')
                          hospitals = update_main_by_average_gnns(hospitals)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN+":
                          hospitals = exchange_models(hospitals, t)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN++":
                          print('4D-FED-GNN++')
                          hospitals = exchange_models_based_on_order_gnns(hospitals, ordered_hospitals)
                      elif epoch % args.D == 0 and args.mode == "weighted_weight_exchange":
                          print('weighted_weight_exchange')
                          hospitals = exchange_models_weights_pairs_extreme(hospitals, t, ordered_hospitals)



            epochs_end = time.time()-epochs_start
            print()
            print(f'epochs finished with time:{epochs_end}')
            print()
            validate_gnns(args, hospitals, test_data, f)
            tic1 = timeit.default_timer()
            timer(tic0,tic1)




In [ ]:
# seed = 10 table = 4/8 fixed folds
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='GAT_4D-FED-GNN++'
train_gnns_final(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
GAT
GAT
GAT
GAT
Epoch:1,val_loss:
Total MAE Loss
[[0.06554328 0.06549242]
 [0.0786098  0.07541355]
 [0.07372822 0.08049995]
 [0.08754892 0.06277961]]

Average MAE Loss:
[0.06551785 0.07701167 0.07711408 0.07516426]

Epoch:2,val_loss:
Total MAE Loss
[[0.06488417 0.06339394]
 [0.07860979 0.07541355]
 [0.06911475 0.07178723]
 [0.08752068 0.06818917]]

Average MAE Loss:
[0.06413906 0.07701167 0.07045099 0.07785493]

Epoch:3,val_loss:
Total MAE Loss
[[0.06155229 0.06012526]
 [0.0786098  0.07541355]
 [0.06904044 0.070440

In [ ]:
# seed = 10 table = 2/8 fixed folds
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/2_8/global_test/GNNs/'
args.save_name='GAT_4D-FED-GNN++'
train_gnns_final(args, dataset,seed=10,ratio=2/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.25

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
GAT
GAT
GAT
GAT
Epoch:1,val_loss:
Total MAE Loss
[[0.0731893  0.07081092]
 [0.0786098  0.07541355]
 [0.06903275 0.06734133]
 [0.08754892 0.06189463]]

Average MAE Loss:
[0.07200011 0.07701167 0.06818704 0.07472178]

Epoch:2,val_loss:
Total MAE Loss
[[0.0731893  0.07081092]
 [0.0786098  0.07541355]
 [0.06861527 0.07096716]
 [0.08754892 0.06226186]]

Average MAE Loss:
[0.07200011 0.07701167 0.06979122 0.07490539]

Epoch:3,val_loss:
Total MAE Loss
[[0.0731893  0.07081092]
 [0.0786098  0.07541355]
 [0.0668805  0.06951

In [ ]:
from google.colab import runtime
runtime.unassign()

## ChebNet


In [ ]:
from torch_geometric.nn.conv import ChebConv

In [ ]:
class DeepChebNet(torch.nn.Module):
    def __init__(self,device):
        super(DeepChebNet, self).__init__()
        in_features = 35
        out_features = 140
        self.conv1 = ChebConv(in_features, out_features, K=2)
        self.conv2 = ChebConv(out_features, 2*out_features, K=2)
        self.fc = nn.Linear(2*out_features,35)
        self.device = device

    def forward(self, adj_matrix):

        data = adj_matrix_to_pytorch_geometric_data(adj_matrix,self.device)
        edge_index = data.edge_index
        data = self.conv1(data.adj_matrix, edge_index)
        data = F.elu(data)
        data = F.dropout(data, p=0.3, training=self.training)
        data = self.conv2(data, edge_index)
        data = F.elu(data)
        data = F.dropout(data, p=0.5, training=self.training)
        out = F.relu(self.fc(data))

        return out

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
data = dataset[0][0].to(device)
gat = DeepChebNet(device).to(device)
gat(data)

tensor([[0.0000, 0.0142, 0.0430,  ..., 0.0000, 0.0237, 0.1471],
        [0.0748, 0.0300, 0.0344,  ..., 0.0000, 0.0000, 0.0879],
        [0.1373, 0.0453, 0.0000,  ..., 0.0000, 0.0000, 0.0661],
        ...,
        [0.0000, 0.0000, 0.1067,  ..., 0.0000, 0.0000, 0.0389],
        [0.0300, 0.0151, 0.0690,  ..., 0.0031, 0.0000, 0.0898],
        [0.1029, 0.0396, 0.0046,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)

In [ ]:
model = DeepChebNet(device)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total parameters:", total_params)
print("Trainable parameters:", trainable_params)

Total parameters: 98455
Trainable parameters: 98455


In [ ]:
class Hospital_gnns():
    def __init__(self, args,device):
        """
        Hospital object contains a GNN and an optimizer for each timepoint

        Hospital object can update GNN-layer wise weights of its GNNs
        """

        self.model = DeepChebNet(device=device).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min',factor=0.5,patience=3,verbose=True)

In [ ]:
def train_gnns_final(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1):
        """
            Arguments:
            args: arguments
            dataset: the whole dataset (train and test set)
            table: [num_hospitals, num_timepoints], holds timepoint-wise availability of hospitals

        This function performs training and testing reporting Mean Absolute Error (MAE) of the testing brain graphs.
        """

        # Create the results folders
        print('Train NEW')
        os.makedirs(args.save_path, exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/real_and_predicted_graphs', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/tp_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/total_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/test_mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/trained_models', exist_ok=True)


        # Create the results folders
        manualSeed = seed

        np.random.seed(manualSeed)
        random.seed(manualSeed)
        torch.manual_seed(manualSeed)

        # show the fixed seed
        print(f'Fixed seed:{seed}')
        print()

        # create the table that shows the data availability by  timepoint
        table = np.zeros((args.num_folds - 1, args.num_timepoints))
        table = random_table(args, ratio)
        print(f'Table:')
        print(table)
        print(f'Ratio:{ratio}')
        print()

        if torch.cuda.is_available():
            device = torch.device('cuda:0')
            print('running on GPU')
            # if you are using GPU
            torch.cuda.manual_seed(manualSeed)
            torch.cuda.manual_seed_all(manualSeed)

            torch.backends.cudnn.enabled = False
            torch.backends.cudnn.benchmark = False
            torch.backends.cudnn.deterministic = False
            os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' # !!! necessary for the below line
            torch.use_deterministic_algorithms(False)
            print('TRAIN deterministic algorithms')

        else:
            device = torch.device("cpu")
            print('running on CPU')

        # change the save path
        args.save_path = args.save_path+f'{args.save_name}/'

        # Choosing the right get_order function
        if args.mode == 'weighted_weight_exchange':
            get_order =  get_order_weighted
        else:
            get_order =  get_order_gnns


        # Getting our fold dict
        fold_dict,X = mf.create_fold_dict_new(dataset,num_hospitals=4,num_folds=5)

        # Perform the 5-fold Cross-Validation
        num_hospitals = args.num_folds - 1
        for f in range(args.num_folds):

            # fix the seeds
            np.random.seed(manualSeed)
            random.seed(manualSeed)
            torch.manual_seed(manualSeed)

            tic0 = timeit.default_timer()

            print(
                f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')

                        # Create hospitals
            hospitals = []
            for i in range(num_hospitals):
                  hospitals.append(Hospital_gnns(args,device))
                  print('DeepChebNet')


            # Train data for each hospital
            train_data_list = []

            # Current test data
            test_data = X[-1][f].to(device)

            for i in range(num_hospitals):

                # Append the fold related to Hospital i - fold f. Note, here we append the real data, not the indices
                train_data_list.append(X[i][fold_dict[f'Hospital_{i}'][f'fold_{f}'][0]].to(device))


            # Start measuring the epochs time
            epochs_start = time.time()

            # Initiate Training
            for epoch in range(args.num_epochs):

                  epoch +=1

                  # order the hospitals based on the data availability
                  ordered_hospitals = get_order_gnns(table)

                  for h_i,hospital in enumerate(hospitals):

                    # get the train data for the hospital
                    train_data = train_data_list[h_i]

                    # get the table for th current hospital
                    table_hospital = table[h_i]

                    # Train the current hospital at the current timepoint for 1 epoch
                    mae_loss,hospital = train_one_epoch_gnns(args,hospital,train_data,table_hospital)

                    # Updating the hospital
                    hospitals[h_i] = hospital

                    if verbose:
                        print(f'Epoch:{epoch} Hospital:{h_i}, Train MAE Loss:{mae_loss}')


                  # Perform validation during training
                  if train_validate_verbose and (epoch%train_validate_verbosity_epochs==0 or epoch==1):

                      val_loss,val_mean_loss = validate_during_training_gnns(args, hospitals, test_data)
                      print(f"Epoch:{epoch},val_loss:")
                      print(f'Total MAE Loss')
                      print(val_loss)
                      print()
                      print(f'Average MAE Loss:')
                      print(val_mean_loss)
                      print()

                      for h_i,l in enumerate(val_mean_loss):

                          hospitals[h_i].scheduler.step(l)


                  if epoch != args.num_epochs - 1 or epoch != 0:
                      if epoch % args.C == 0 and args.mode != "4D-GNN":
                          print('Central Aggregation')
                          hospitals = update_main_by_average_gnns(hospitals)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN+":
                          hospitals = exchange_models(hospitals, t)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN++":
                          print('4D-FED-GNN++')
                          hospitals = exchange_models_based_on_order_gnns(hospitals, ordered_hospitals)
                      elif epoch % args.D == 0 and args.mode == "weighted_weight_exchange":
                          print('weighted_weight_exchange')
                          hospitals = exchange_models_weights_pairs_extreme(hospitals, t, ordered_hospitals)



            epochs_end = time.time()-epochs_start
            print()
            print(f'epochs finished with time:{epochs_end}')
            print()
            validate_gnns(args, hospitals, test_data, f)
            tic1 = timeit.default_timer()
            timer(tic0,tic1)




In [ ]:
# seed = 10 table = 4/8 fixed folds 3 layers adj
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='Cheb_4D-FED-GNN++'
train_gnns_final(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
DeepChebNet
DeepChebNet
DeepChebNet
DeepChebNet
Epoch:1,val_loss:
Total MAE Loss
[[0.04572127 0.0473866 ]
 [0.07827297 0.07513477]
 [0.04737947 0.0493499 ]
 [0.07588218 0.06153284]]

Average MAE Loss:
[0.04655393 0.07670387 0.04836469 0.06870751]

Epoch:2,val_loss:
Total MAE Loss
[[0.03840624 0.03943488]
 [0.07827297 0.07513476]
 [0.04450329 0.04762704]
 [0.07928729 0.05333582]]

Average MAE Loss:
[0.03892056 0.07670387 0.04606516 0.06631155]

Epoch:3,val_loss:
Total MAE Loss
[[0.03776373 0.03822628]
 [0.07827297 0

In [ ]:
# seed = 10 table = 2/8 fixed folds 3 layers adj
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/2_8/global_test/GNNs/'
args.save_name='Cheb_4D-FED-GNN++'
train_gnns_final(args, dataset,seed=10,ratio=2/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.25

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
DeepChebNet
DeepChebNet
DeepChebNet
DeepChebNet
Epoch:1,val_loss:
Total MAE Loss
[[0.07840958 0.07396808]
 [0.07827297 0.07513475]
 [0.04662903 0.04930476]
 [0.07608913 0.06248086]]

Average MAE Loss:
[0.07618883 0.07670386 0.0479669  0.069285  ]

Epoch:2,val_loss:
Total MAE Loss
[[0.07840958 0.07396808]
 [0.07827297 0.07513475]
 [0.042727   0.04538441]
 [0.07781408 0.05591736]]

Average MAE Loss:
[0.07618883 0.07670386 0.0440557  0.06686572]

Epoch:3,val_loss:
Total MAE Loss
[[0.07840958 0.07396808]
 [0.07827297 

## TAGConv

In [ ]:
from torch_geometric.nn.conv import TAGConv

In [ ]:
class DeepTAGNet(torch.nn.Module):
    def __init__(self,device):
        super(DeepTAGNet, self).__init__()
        in_features = 35
        out_features = 140
        self.conv1 = TAGConv(in_features, out_features)
        self.conv2 = TAGConv(out_features, 2*out_features)
        self.fc1 = nn.Linear(2*out_features,35)
        self.device = device

    def forward(self, adj_matrix):

        data = adj_matrix_to_pytorch_geometric_data(adj_matrix,self.device)
        edge_index = data.edge_index
        data = self.conv1(data.adj_matrix, edge_index)
        data = F.elu(data)
        data = F.dropout(data, p=0.3, training=self.training)

        data = self.conv2(data, edge_index)
        data = F.elu(data)
        data = F.dropout(data, p=0.5, training=self.training)

        out = F.relu(self.fc1(data))


        return out

In [ ]:
model = DeepTAGNet(device).to(device)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total parameters:", total_params)
print("Trainable parameters:", trainable_params)

Total parameters: 186655
Trainable parameters: 186655


In [ ]:
model(dataset[0][0]).shape

torch.Size([35, 35])

In [ ]:
class Hospital_gnns():
    def __init__(self, args,device):
        """
        Hospital object contains a GNN and an optimizer for each timepoint

        Hospital object can update GNN-layer wise weights of its GNNs
        """

        self.model = DeepTAGNet(device=device).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min',factor=0.5,patience=3,verbose=True)

In [ ]:
def train_gnns_final(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1):
        """
            Arguments:
            args: arguments
            dataset: the whole dataset (train and test set)
            table: [num_hospitals, num_timepoints], holds timepoint-wise availability of hospitals

        This function performs training and testing reporting Mean Absolute Error (MAE) of the testing brain graphs.
        """

        # Create the results folders
        print('Train NEW')
        os.makedirs(args.save_path, exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/real_and_predicted_graphs', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/tp_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/total_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/test_mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/trained_models', exist_ok=True)


        # Create the results folders
        manualSeed = seed

        np.random.seed(manualSeed)
        random.seed(manualSeed)
        torch.manual_seed(manualSeed)

        # show the fixed seed
        print(f'Fixed seed:{seed}')
        print()

        # create the table that shows the data availability by  timepoint
        table = np.zeros((args.num_folds - 1, args.num_timepoints))
        table = random_table(args, ratio)
        print(f'Table:')
        print(table)
        print(f'Ratio:{ratio}')
        print()

        if torch.cuda.is_available():
            device = torch.device('cuda:0')
            print('running on GPU')
            # if you are using GPU
            torch.cuda.manual_seed(manualSeed)
            torch.cuda.manual_seed_all(manualSeed)

            torch.backends.cudnn.enabled = False
            torch.backends.cudnn.benchmark = False
            torch.backends.cudnn.deterministic = False
            os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' # !!! necessary for the below line
            torch.use_deterministic_algorithms(False)
            print('TRAIN deterministic algorithms')

        else:
            device = torch.device("cpu")
            print('running on CPU')

        # change the save path
        args.save_path = args.save_path+f'{args.save_name}/'

        # Choosing the right get_order function
        if args.mode == 'weighted_weight_exchange':
            get_order =  get_order_weighted
        else:
            get_order =  get_order_gnns


        # Getting our fold dict
        fold_dict,X = mf.create_fold_dict_new(dataset,num_hospitals=4,num_folds=5)

        # Perform the 5-fold Cross-Validation
        num_hospitals = args.num_folds - 1
        for f in range(args.num_folds):

            # fix the seeds
            np.random.seed(manualSeed)
            random.seed(manualSeed)
            torch.manual_seed(manualSeed)

            tic0 = timeit.default_timer()

            print(
                f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')

                        # Create hospitals
            hospitals = []
            for i in range(num_hospitals):
                  hospitals.append(Hospital_gnns(args,device))
                  print('DeepTAGNet')


            # Train data for each hospital
            train_data_list = []

            # Current test data
            test_data = X[-1][f].to(device)

            for i in range(num_hospitals):

                # Append the fold related to Hospital i - fold f. Note, here we append the real data, not the indices
                train_data_list.append(X[i][fold_dict[f'Hospital_{i}'][f'fold_{f}'][0]].to(device))


            # Start measuring the epochs time
            epochs_start = time.time()

            # Initiate Training
            for epoch in range(args.num_epochs):

                  epoch +=1

                  # order the hospitals based on the data availability
                  ordered_hospitals = get_order_gnns(table)

                  for h_i,hospital in enumerate(hospitals):

                    # get the train data for the hospital
                    train_data = train_data_list[h_i]

                    # get the table for th current hospital
                    table_hospital = table[h_i]

                    # Train the current hospital at the current timepoint for 1 epoch
                    mae_loss,hospital = train_one_epoch_gnns(args,hospital,train_data,table_hospital)

                    # Updating the hospital
                    hospitals[h_i] = hospital

                    if verbose:
                        print(f'Epoch:{epoch} Hospital:{h_i}, Train MAE Loss:{mae_loss}')


                  # Perform validation during training
                  if train_validate_verbose and (epoch%train_validate_verbosity_epochs==0 or epoch==1):

                      val_loss,val_mean_loss = validate_during_training_gnns(args, hospitals, test_data)
                      print(f"Epoch:{epoch},val_loss:")
                      print(f'Total MAE Loss')
                      print(val_loss)
                      print()
                      print(f'Average MAE Loss:')
                      print(val_mean_loss)
                      print()

                      for h_i,l in enumerate(val_mean_loss):

                          hospitals[h_i].scheduler.step(l)


                  if epoch != args.num_epochs - 1 or epoch != 0:
                      if epoch % args.C == 0 and args.mode != "4D-GNN":
                          print('Central Aggregation')
                          hospitals = update_main_by_average_gnns(hospitals)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN+":
                          hospitals = exchange_models(hospitals, t)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN++":
                          print('4D-FED-GNN++')
                          hospitals = exchange_models_based_on_order_gnns(hospitals, ordered_hospitals)
                      elif epoch % args.D == 0 and args.mode == "weighted_weight_exchange":
                          print('weighted_weight_exchange')
                          hospitals = exchange_models_weights_pairs_extreme(hospitals, t, ordered_hospitals)



            epochs_end = time.time()-epochs_start
            print()
            print(f'epochs finished with time:{epochs_end}')
            print()
            validate_gnns(args, hospitals, test_data, f)
            tic1 = timeit.default_timer()
            timer(tic0,tic1)




In [ ]:
# seed = 10 table = 4/8 fixed folds 2+1 layers adj 140
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='TAG_4D-FED-GNN++'
train_gnns_final(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
DeepTAGNet
DeepTAGNet
DeepTAGNet
DeepTAGNet
Epoch:1,val_loss:
Total MAE Loss
[[0.04718883 0.05323763]
 [0.0762185  0.07197651]
 [0.05193045 0.05785387]
 [0.08247099 0.06363742]]

Average MAE Loss:
[0.05021323 0.07409751 0.05489216 0.0730542 ]

Epoch:2,val_loss:
Total MAE Loss
[[0.04289958 0.04841904]
 [0.0762185  0.07197651]
 [0.04429915 0.04774465]
 [0.08677099 0.05083821]]

Average MAE Loss:
[0.04565931 0.0740975  0.0460219  0.0688046 ]

Epoch:3,val_loss:
Total MAE Loss
[[0.03921052 0.04293645]
 [0.0762185  0.071

In [ ]:
# seed = 10 table = 2/8 fixed folds 2+1 layers adj 140
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/2_8/global_test/GNNs/'
args.save_name='TAG_4D-FED-GNN++'
train_gnns_final(args, dataset,seed=10,ratio=2/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

## GAT dream

In [ ]:
from torch_geometric.nn import GATConv

In [ ]:
class GAT_dream(torch.nn.Module):
    def __init__(self,device):
        super(GAT_dream, self).__init__()
        self.in_channels = 10
        self.hidden_channels = 64
        self.heads=8
        self.out_head = 1
        self.device=device
        self.num_nodes = 35


        self.conv1 = GATConv(self.in_channels, self.hidden_channels, heads=self.heads, dropout=0.6)
        self.conv2 = GATConv(self.hidden_channels*self.heads, self.hidden_channels, concat=False,
                             heads=self.out_head, dropout=0.6)
        self.fc = nn.Linear(self.hidden_channels,self.num_nodes)
        self.fc_dream = nn.Sequential(nn.Linear(35,10), nn.ELU(), nn.Linear(10,10), nn.ELU())


    def forward(self, adj_matrix):

        # learning suitable node representation from adj matrix
        x = self.fc_dream(adj_matrix)
        data = adj_matrix_to_pytorch_geometric_data(adj_matrix,self.device)
        edge_index = data.edge_index

        x = F.dropout(x, p=0.6, training=self.training)
        data = self.conv1(x, edge_index)
        data = F.elu(data)
        data = F.dropout(data, p=0.6, training=self.training)
        data = self.conv2(data, edge_index)
        out = F.relu(self.fc(data))

        return out

In [ ]:
torch.use_deterministic_algorithms(False)

In [ ]:
data = dataset[0][0].to(device)
gat = GAT_dream(device).to(device)
gat(data).sum()

tensor(34.4247, device='cuda:0', grad_fn=<SumBackward0>)

In [ ]:
model = GAT(device)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total parameters:", total_params)
print("Trainable parameters:", trainable_params)

Total parameters: 42251
Trainable parameters: 42251


In [ ]:
device = torch.device('cuda:0')

In [ ]:
class Hospital_gat_dream():
    def __init__(self, args,device):
        """
        Hospital object contains a GNN and an optimizer for each timepoint

        Hospital object can update GNN-layer wise weights of its GNNs
        """

        self.model = GAT_dream(device=device).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min',factor=0.5,patience=3,verbose=True)

In [ ]:
def train_gnns_final(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1):
        """
            Arguments:
            args: arguments
            dataset: the whole dataset (train and test set)
            table: [num_hospitals, num_timepoints], holds timepoint-wise availability of hospitals

        This function performs training and testing reporting Mean Absolute Error (MAE) of the testing brain graphs.
        """

        # Create the results folders
        print('Train NEW')
        os.makedirs(args.save_path, exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/real_and_predicted_graphs', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/tp_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/total_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/test_mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/trained_models', exist_ok=True)


        # Create the results folders
        manualSeed = seed

        np.random.seed(manualSeed)
        random.seed(manualSeed)
        torch.manual_seed(manualSeed)

        # show the fixed seed
        print(f'Fixed seed:{seed}')
        print()

        # create the table that shows the data availability by  timepoint
        table = np.zeros((args.num_folds - 1, args.num_timepoints))
        table = random_table(args, ratio)
        print(f'Table:')
        print(table)
        print(f'Ratio:{ratio}')
        print()

        if torch.cuda.is_available():
            device = torch.device('cuda:0')
            print('running on GPU')
            # if you are using GPU
            torch.cuda.manual_seed(manualSeed)
            torch.cuda.manual_seed_all(manualSeed)

            torch.backends.cudnn.enabled = False
            torch.backends.cudnn.benchmark = False
            torch.backends.cudnn.deterministic = False
            os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' # !!! necessary for the below line
            torch.use_deterministic_algorithms(False)
            print('TRAIN deterministic algorithms')

        else:
            device = torch.device("cpu")
            print('running on CPU')

        # change the save path
        args.save_path = args.save_path+f'{args.save_name}/'

        # Choosing the right get_order function
        if args.mode == 'weighted_weight_exchange':
            get_order =  get_order_weighted
        else:
            get_order =  get_order_gnns


        # Getting our fold dict
        fold_dict,X = mf.create_fold_dict_new(dataset,num_hospitals=4,num_folds=5)

        # Perform the 5-fold Cross-Validation
        num_hospitals = args.num_folds - 1
        for f in range(args.num_folds):

            # fix the seeds
            np.random.seed(manualSeed)
            random.seed(manualSeed)
            torch.manual_seed(manualSeed)

            tic0 = timeit.default_timer()

            print(
                f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')

                        # Create hospitals
            hospitals = []
            for i in range(num_hospitals):
                  hospitals.append(Hospital_gat_dream(args,device))
                  print('GAT dream')


            # Train data for each hospital
            train_data_list = []

            # Current test data
            test_data = X[-1][f].to(device)

            for i in range(num_hospitals):

                # Append the fold related to Hospital i - fold f. Note, here we append the real data, not the indices
                train_data_list.append(X[i][fold_dict[f'Hospital_{i}'][f'fold_{f}'][0]].to(device))


            # Start measuring the epochs time
            epochs_start = time.time()

            # Initiate Training
            for epoch in range(args.num_epochs):

                  epoch +=1

                  # order the hospitals based on the data availability
                  ordered_hospitals = get_order_gnns(table)

                  for h_i,hospital in enumerate(hospitals):

                    # get the train data for the hospital
                    train_data = train_data_list[h_i]

                    # get the table for th current hospital
                    table_hospital = table[h_i]

                    # Train the current hospital at the current timepoint for 1 epoch
                    mae_loss,hospital = train_one_epoch_gnns(args,hospital,train_data,table_hospital)

                    # Updating the hospital
                    hospitals[h_i] = hospital

                    if verbose:
                        print(f'Epoch:{epoch} Hospital:{h_i}, Train MAE Loss:{mae_loss}')


                  # Perform validation during training
                  if train_validate_verbose and (epoch%train_validate_verbosity_epochs==0 or epoch==1):

                      val_loss,val_mean_loss = validate_during_training_gnns(args, hospitals, test_data)
                      print(f"Epoch:{epoch},val_loss:")
                      print(f'Total MAE Loss')
                      print(val_loss)
                      print()
                      print(f'Average MAE Loss:')
                      print(val_mean_loss)
                      print()

                      for h_i,l in enumerate(val_mean_loss):

                          hospitals[h_i].scheduler.step(l)


                  if epoch != args.num_epochs - 1 or epoch != 0:
                      if epoch % args.C == 0 and args.mode != "4D-GNN":
                          print('Central Aggregation')
                          hospitals = update_main_by_average_gnns(hospitals)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN+":
                          hospitals = exchange_models(hospitals, t)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN++":
                          print('4D-FED-GNN++')
                          hospitals = exchange_models_based_on_order_gnns(hospitals, ordered_hospitals)
                      elif epoch % args.D == 0 and args.mode == "weighted_weight_exchange":
                          print('weighted_weight_exchange')
                          hospitals = exchange_models_weights_pairs_extreme(hospitals, t, ordered_hospitals)



            epochs_end = time.time()-epochs_start
            print()
            print(f'epochs finished with time:{epochs_end}')
            print()
            validate_gnns(args, hospitals, test_data, f)
            tic1 = timeit.default_timer()
            timer(tic0,tic1)




In [ ]:
# table = 4/8 fixed folds
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='GAT_dream_4D-FED-GNN++'
train_gnns_final(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
GAT dream
GAT dream
GAT dream
GAT dream
Epoch:1,val_loss:
Total MAE Loss
[[0.06138949 0.05815226]
 [0.0784388  0.07544469]
 [0.06316184 0.06054214]
 [0.06397959 0.061189  ]]

Average MAE Loss:
[0.05977088 0.07694175 0.06185199 0.06258429]

Epoch:2,val_loss:
Total MAE Loss
[[0.06061132 0.05789622]
 [0.0784388  0.07544469]
 [0.06204481 0.05905726]
 [0.0627647  0.06066238]]

Average MAE Loss:
[0.05925377 0.07694175 0.06055104 0.06171354]

Epoch:3,val_loss:
Total MAE Loss
[[0.05979239 0.05687716]
 [0.0784388  0.0754446

KeyboardInterrupt: ignored

## MetaLayer


In [ ]:
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_geometric.utils import scatter
from torch_geometric.nn import MetaLayer
from typing import Optional, Tuple
from torch import Tensor
from torch_geometric.utils import dense_to_sparse
import networkx as nx

In [ ]:
def get_adjacency_matrix(num_nodes, edge_indices, edge_weights,device):
    # Initialize an empty adjacency matrix
    adjacency_matrix = torch.zeros((num_nodes, num_nodes))

    # Fill in the adjacency matrix
    for ((node1, node2), weight) in zip(edge_indices.T, edge_weights):
        adjacency_matrix[node1, node2] = weight

    return adjacency_matrix.to(device)

In [ ]:
class MetaLayer(torch.nn.Module):
    r"""A meta layer for building any kind of graph network, inspired by the
    `"Relational Inductive Biases, Deep Learning, and Graph Networks"
    <https://arxiv.org/abs/1806.01261>`_ paper.

    A graph network takes a graph as input and returns an updated graph as
    output (with same connectivity).
    The input graph has node features :obj:`x`, edge features :obj:`edge_attr`
    as well as graph-level features :obj:`u`.
    The output graph has the same structure, but updated features.

    Edge features, node features as well as global features are updated by
    calling the modules :obj:`edge_model`, :obj:`node_model` and
    :obj:`global_model`, respectively.

    To allow for batch-wise graph processing, all callable functions take an
    additional argument :obj:`batch`, which determines the assignment of
    edges or nodes to their specific graphs.

    Args:
        edge_model (torch.nn.Module, optional): A callable which updates a
            graph's edge features based on its source and target node features,
            its current edge features and its global features.
            (default: :obj:`None`)
        node_model (torch.nn.Module, optional): A callable which updates a
            graph's node features based on its current node features, its graph
            connectivity, its edge features and its global features.
            (default: :obj:`None`)
        global_model (torch.nn.Module, optional): A callable which updates a
            graph's global features based on its node features, its graph
            connectivity, its edge features and its current global features.
            (default: :obj:`None`)

    .. code-block:: python

        from torch.nn import Sequential as Seq, Linear as Lin, ReLU
        from torch_geometric.utils import scatter
        from torch_geometric.nn import MetaLayer

        class EdgeModel(torch.nn.Module):
            def __init__(self):
                super().__init__()
                self.edge_mlp = Seq(Lin(..., ...), ReLU(), Lin(..., ...))

            def forward(self, src, dst, edge_attr, u, batch):
                # src, dst: [E, F_x], where E is the number of edges.
                # edge_attr: [E, F_e]
                # u: [B, F_u], where B is the number of graphs.
                # batch: [E] with max entry B - 1.
                out = torch.cat([src, dst, edge_attr, u[batch]], 1)
                return self.edge_mlp(out)

        class NodeModel(torch.nn.Module):
            def __init__(self):
                super().__init__()
                self.node_mlp_1 = Seq(Lin(..., ...), ReLU(), Lin(..., ...))
                self.node_mlp_2 = Seq(Lin(..., ...), ReLU(), Lin(..., ...))

            def forward(self, x, edge_index, edge_attr, u, batch):
                # x: [N, F_x], where N is the number of nodes.
                # edge_index: [2, E] with max entry N - 1.
                # edge_attr: [E, F_e]
                # u: [B, F_u]
                # batch: [N] with max entry B - 1.
                row, col = edge_index
                out = torch.cat([x[row], edge_attr], dim=1)
                out = self.node_mlp_1(out)
                out = scatter(out, col, dim=0, dim_size=x.size(0),
                              reduce='mean')
                out = torch.cat([x, out, u[batch]], dim=1)
                return self.node_mlp_2(out)

        class GlobalModel(torch.nn.Module):
            def __init__(self):
                super().__init__()
                self.global_mlp = Seq(Lin(..., ...), ReLU(), Lin(..., ...))

            def forward(self, x, edge_index, edge_attr, u, batch):
                # x: [N, F_x], where N is the number of nodes.
                # edge_index: [2, E] with max entry N - 1.
                # edge_attr: [E, F_e]
                # u: [B, F_u]
                # batch: [N] with max entry B - 1.
                out = torch.cat([
                    u,
                    scatter(x, batch, dim=0, reduce='mean'),
                ], dim=1)
                return self.global_mlp(out)

        op = MetaLayer(EdgeModel(), NodeModel(), GlobalModel())
        x, edge_attr, u = op(x, edge_index, edge_attr, u, batch)
    """
    def __init__(
        self,
        edge_model: Optional[torch.nn.Module] = None,
        node_model: Optional[torch.nn.Module] = None,
        global_model: Optional[torch.nn.Module] = None,
    ):
        super().__init__()
        self.edge_model = edge_model
        self.node_model = node_model
        self.global_model = global_model

        self.reset_parameters()

    def reset_parameters(self):
        r"""Resets all learnable parameters of the module."""
        for item in [self.node_model, self.edge_model, self.global_model]:
            if hasattr(item, 'reset_parameters'):
                item.reset_parameters()


    def forward(
        self,
        x: Tensor,
        edge_index: Tensor,
        edge_attr: Optional[Tensor] = None,
        u: Optional[Tensor] = None,
        batch: Optional[Tensor] = None,
    ) -> Tuple[Tensor, Optional[Tensor], Optional[Tensor]]:
        r"""
        Args:
            x (torch.Tensor): The node features.
            edge_index (torch.Tensor): The edge indices.
            edge_attr (torch.Tensor, optional): The edge features.
                (default: :obj:`None`)
            u (torch.Tensor, optional): The global graph features.
                (default: :obj:`None`)
            batch (torch.Tensor, optional): The batch vector
                :math:`\mathbf{b} \in {\{ 0, \ldots, B-1\}}^N`, which assigns
                each node to a specific graph. (default: :obj:`None`)
        """
        row = edge_index[0]
        col = edge_index[1]

        if self.edge_model is not None:
            edge_attr = self.edge_model(x[row], x[col], edge_attr)


        if self.node_model is not None:
            x = self.node_model(x, edge_index, edge_attr)

        if self.global_model is not None:
            u = self.global_model(x, edge_index, edge_attr, u, batch)

        return x, edge_attr, u


    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}(\n'
                f'  edge_model={self.edge_model},\n'
                f'  node_model={self.node_model},\n'
                f'  global_model={self.global_model}\n'
                f')')

In [ ]:
class EdgeModel(torch.nn.Module):
    def __init__(self,input=11,out1=15,out2=20):
        super().__init__()
        self.edge_mlp = Seq(Lin(input, out1), nn.ELU(), Lin(out1, out2))

    def forward(self, src, dst, edge_attr):
        # src, dst: [E, F_x], where E is the number of edges.
        # edge_attr: [E, F_e]
        # u: [B, F_u], where B is the number of graphs.
        # batch: [E] with max entry B - 1.
        out = torch.cat([src, dst, edge_attr], 1)
        return self.edge_mlp(out)

class NodeModel(torch.nn.Module):
    def __init__(self,input1,input2,out11,out12,out21,out22):
        super().__init__()
        self.node_mlp_1 = Seq(Lin(input1, out11), nn.ELU(), Lin(out11, out12))
        self.node_mlp_2 = Seq(Lin(input2, out21), nn.Sigmoid(), Lin(out21, out22))

    def forward(self, x, edge_index, edge_attr):
        # x: [N, F_x], where N is the number of nodes.
        # edge_index: [2, E] with max entry N - 1.
        # edge_attr: [E, F_e]
        # u: [B, F_u]
        # batch: [N] with max entry B - 1.
        row, col = edge_index
        out = torch.cat([x[row], edge_attr], dim=1)
        out = self.node_mlp_1(out)
        out = scatter(out, col, dim=0, dim_size=x.size(0),
                      reduce='mean')
        out = torch.cat([x, out], dim=1)
        return self.node_mlp_2(out)

In [ ]:
class DeepMetaLayer(torch.nn.Module):
    def __init__(self,device):
        super().__init__()
        self.meta1 = MetaLayer(EdgeModel(input=13,out1=18,out2=20),NodeModel(input1=20+5,input2=5+35,out11=30,out12=35,out21=40,out22=45))
        self.meta2 = MetaLayer(EdgeModel(input=20+45*2,out1=120,out2=130),NodeModel(input1=130+45,input2=45+1,out11=1,out12=1,out21=1,out22=1))
        self.fc1 = nn.Linear(130,5)
        self.fc2 = nn.Linear(5,1)


        self.device = device
    def forward(self, adj_matrix):


        data = adj_matrix_to_pytorch_geometric_data(adj_matrix,self.device)
        x, edge_attr, u = self.meta1(data.x, data.edge_index, data.edge_attr)
        #print(edge_attr)
        x, edge_attr, u = self.meta2(x, data.edge_index, edge_attr)
        edge_attr = edge_attr
        #print(edge_attr)
        edge_weights = F.elu(self.fc1(edge_attr))
        #print(edge_weights)
        #print(edge_weights.shape)
        #print(edge_weights.sum())

        edge_weights = F.sigmoid(self.fc2(edge_weights))
        #print(edge_weights)
        #print(edge_weights.shape)
        #print(edge_weights.sum())

        out = get_adjacency_matrix(35, data.edge_index, edge_weights,device)


        return out



In [ ]:
class DeepMetaLayer_adj(torch.nn.Module):
    def __init__(self,device):
        super().__init__()
        self.meta1 = MetaLayer(EdgeModel(input=70+3,out1=80,out2=80),NodeModel(input1=35+80,input2=35+115,out11=115,out12=115,out21=125,out22=125))
        self.meta2 = MetaLayer(EdgeModel(input=80+125*2,out1=350,out2=350),NodeModel(input1=125+350,input2=125+1,out11=1,out12=1,out21=1,out22=1))
        self.fc1 = nn.Linear(350,50)
        self.fc2 = nn.Linear(50,1)


        self.device = device
    def forward(self, adj_matrix):


        data = adj_matrix_to_pytorch_geometric_data(adj_matrix,self.device)
        x, edge_attr, u = self.meta1(data.adj_matrix, data.edge_index, data.edge_attr)
        #print(edge_attr)
        x, edge_attr, u = self.meta2(x, data.edge_index, edge_attr)
        edge_attr = edge_attr
        #print(edge_attr)
        edge_weights = F.elu(self.fc1(edge_attr))
        #print(edge_weights)
        #print(edge_weights.shape)
        #print(edge_weights.sum())

        edge_weights = F.sigmoid(self.fc2(edge_weights))
        #print(edge_weights)
        #print(edge_weights.shape)
        #print(edge_weights.sum())

        out = get_adjacency_matrix(35, data.edge_index, edge_weights,device)


        return out

In [ ]:
data = dataset[0][0]

In [ ]:
model =  DeepMetaLayer(device).to(device)
out = model(data)
print(out)
print(out.sum())

tensor([[0.0000, 0.5725, 0.5738,  ..., 0.0000, 0.6195, 0.6206],
        [0.5688, 0.0000, 0.5726,  ..., 0.6172, 0.6191, 0.6201],
        [0.5687, 0.5709, 0.0000,  ..., 0.6171, 0.6190, 0.6200],
        ...,
        [0.0000, 0.5920, 0.5929,  ..., 0.0000, 0.6351, 0.6361],
        [0.5916, 0.5926, 0.5935,  ..., 0.6345, 0.0000, 0.6367],
        [0.5922, 0.5932, 0.5941,  ..., 0.6349, 0.6361, 0.0000]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor(711.3232, device='cuda:0', grad_fn=<SumBackward0>)


In [ ]:
class Hospital_meta():
    def __init__(self, args,device):
        """
        Hospital object contains a GNN and an optimizer for each timepoint

        Hospital object can update GNN-layer wise weights of its GNNs
        """

        self.model =  DeepMetaLayer(device).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min',factor=0.5,patience=3,verbose=True)

In [ ]:
model = DeepMetaLayer(device).to(device)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total parameters:", total_params)
print("Trainable parameters:", trainable_params)

Total parameters: 35920
Trainable parameters: 35920


In [ ]:
def train_gnns_final(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1):
        """
            Arguments:
            args: arguments
            dataset: the whole dataset (train and test set)
            table: [num_hospitals, num_timepoints], holds timepoint-wise availability of hospitals

        This function performs training and testing reporting Mean Absolute Error (MAE) of the testing brain graphs.
        """

        # Create the results folders
        print('Train NEW')
        os.makedirs(args.save_path, exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/real_and_predicted_graphs', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/tp_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/total_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/test_mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/trained_models', exist_ok=True)


        # Create the results folders
        manualSeed = seed

        np.random.seed(manualSeed)
        random.seed(manualSeed)
        torch.manual_seed(manualSeed)

        # show the fixed seed
        print(f'Fixed seed:{seed}')
        print()

        # create the table that shows the data availability by  timepoint
        table = np.zeros((args.num_folds - 1, args.num_timepoints))
        table = random_table(args, ratio)
        print(f'Table:')
        print(table)
        print(f'Ratio:{ratio}')
        print()

        if torch.cuda.is_available():
            device = torch.device('cuda:0')
            print('running on GPU')
            # if you are using GPU
            torch.cuda.manual_seed(manualSeed)
            torch.cuda.manual_seed_all(manualSeed)

            torch.backends.cudnn.enabled = False
            torch.backends.cudnn.benchmark = False
            torch.backends.cudnn.deterministic = False
            os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' # !!! necessary for the below line
            torch.use_deterministic_algorithms(False)
            print('TRAIN deterministic algorithms')

        else:
            device = torch.device("cpu")
            print('running on CPU')

        # change the save path
        args.save_path = args.save_path+f'{args.save_name}/'

        # Choosing the right get_order function
        if args.mode == 'weighted_weight_exchange':
            get_order =  get_order_weighted
        else:
            get_order =  get_order_gnns


        # Getting our fold dict
        fold_dict,X = mf.create_fold_dict_new(dataset,num_hospitals=4,num_folds=5)

        # Perform the 5-fold Cross-Validation
        num_hospitals = args.num_folds - 1
        for f in range(args.num_folds):

            # fix the seeds
            np.random.seed(manualSeed)
            random.seed(manualSeed)
            torch.manual_seed(manualSeed)

            tic0 = timeit.default_timer()

            print(
                f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')

                        # Create hospitals
            hospitals = []
            for i in range(num_hospitals):
                  hospitals.append(Hospital_meta(args,device))
                  print('Meta 2 X')


            # Train data for each hospital
            train_data_list = []

            # Current test data
            test_data = X[-1][f].to(device)

            for i in range(num_hospitals):

                # Append the fold related to Hospital i - fold f. Note, here we append the real data, not the indices
                train_data_list.append(X[i][fold_dict[f'Hospital_{i}'][f'fold_{f}'][0]].to(device))


            # Start measuring the epochs time
            epochs_start = time.time()

            # Initiate Training
            for epoch in range(args.num_epochs):

                  epoch +=1

                  # order the hospitals based on the data availability
                  ordered_hospitals = get_order_gnns(table)

                  for h_i,hospital in enumerate(hospitals):

                    # get the train data for the hospital
                    train_data = train_data_list[h_i]

                    # get the table for th current hospital
                    table_hospital = table[h_i]

                    #if h_i ==3:
                     # return args,hospital,train_data,table_hospital

                    # Train the current hospital at the current timepoint for 1 epoch
                    mae_loss,hospital = train_one_epoch_gnns(args,hospital,train_data,table_hospital)

                    # Updating the hospital
                    hospitals[h_i] = hospital

                    if verbose:
                        print(f'Epoch:{epoch} Hospital:{h_i}, Train MAE Loss:{mae_loss}')


                  # Perform validation during training
                  if train_validate_verbose and (epoch%train_validate_verbosity_epochs==0 or epoch==1):

                      val_loss,val_mean_loss = validate_during_training_gnns(args, hospitals, test_data)
                      print(f"Epoch:{epoch},val_loss:")
                      print(f'Total MAE Loss')
                      print(val_loss)
                      print()
                      print(f'Average MAE Loss:')
                      print(val_mean_loss)
                      print()

                      for h_i,l in enumerate(val_mean_loss):

                          hospitals[h_i].scheduler.step(l)


                  if epoch != args.num_epochs - 1 or epoch != 0:
                      if epoch % args.C == 0 and args.mode != "4D-GNN":
                          print('Central Aggregation')
                          hospitals = update_main_by_average_gnns(hospitals)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN+":
                          hospitals = exchange_models(hospitals, t)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN++":
                          print('4D-FED-GNN++')
                          hospitals = exchange_models_based_on_order_gnns(hospitals, ordered_hospitals)
                      elif epoch % args.D == 0 and args.mode == "weighted_weight_exchange":
                          print('weighted_weight_exchange')
                          hospitals = exchange_models_weights_pairs_extreme(hospitals, t, ordered_hospitals)



            epochs_end = time.time()-epochs_start
            print()
            print(f'epochs finished with time:{epochs_end}')
            print()
            validate_gnns(args, hospitals, test_data, f)
            tic1 = timeit.default_timer()
            timer(tic0,tic1)




In [ ]:
# seed = 10 table = 4/8 fixed Meta 2 layers x
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='Meta_4D-FED-GNN++'
train_gnns_final(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Meta 2 X
Meta 2 X
Meta 2 X
Meta 2 X
Epoch:1,val_loss:
Total MAE Loss
[[0.05345583 0.05097473]
 [0.49095011 0.49543648]
 [0.06533224 0.06207772]
 [0.06643013 0.06219492]]

Average MAE Loss:
[0.05221528 0.49319329 0.06370498 0.06431252]

Epoch:2,val_loss:
Total MAE Loss
[[0.05415963 0.05153574]
 [0.49095011 0.49543648]
 [0.05382052 0.05131982]
 [0.05640134 0.05298167]]

Average MAE Loss:
[0.05284768 0.49319329 0.05257017 0.05469151]

Epoch:3,val_loss:
Total MAE Loss
[[0.05318925 0.05082954]
 [0.49095011 0.49543648]
 

TypeError: ignored

In [ ]:
# seed = 10 table = 2/8 fixed Meta 2 layers x
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/2_8/global_test/GNNs/'
args.save_name='Meta_4D-FED-GNN++'
train_gnns_final(args, dataset,seed=10,ratio=2/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.25

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Meta 2 X
Meta 2 X
Meta 2 X
Meta 2 X
Epoch:1,val_loss:
Total MAE Loss
[[0.22951756 0.22984366]
 [0.49095011 0.49543648]
 [0.06533224 0.06207772]
 [0.06643012 0.06219492]]

Average MAE Loss:
[0.22968061 0.49319329 0.06370498 0.06431252]

Epoch:2,val_loss:
Total MAE Loss
[[0.22951756 0.22984366]
 [0.49095011 0.49543648]
 [0.05382052 0.05131982]
 [0.05640135 0.05298167]]

Average MAE Loss:
[0.22968061 0.49319329 0.05257017 0.05469151]

Epoch:3,val_loss:
Total MAE Loss
[[0.22951756 0.22984366]
 [0.49095011 0.49543648]


In [ ]:
from google.colab import runtime
runtime.unassign()

## CNNs

In [ ]:
def train_cnns(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1):
        """
            Arguments:
            args: arguments
            dataset: the whole dataset (train and test set)
            table: [num_hospitals, num_timepoints], holds timepoint-wise availability of hospitals

        This function performs training and testing reporting Mean Absolute Error (MAE) of the testing brain graphs.
        """

        # Create the results folders
        print('Train NEW')
        os.makedirs(args.save_path, exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/real_and_predicted_graphs', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/tp_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/total_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/test_mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/trained_models', exist_ok=True)


        # Create the results folders
        manualSeed = seed

        np.random.seed(manualSeed)
        random.seed(manualSeed)
        torch.manual_seed(manualSeed)

        # show the fixed seed
        print(f'Fixed seed:{seed}')
        print()

        # create the table that shows the data availability by  timepoint
        table = np.zeros((args.num_folds - 1, args.num_timepoints))
        table = random_table(args, ratio)
        print(f'Table:')
        print(table)
        print(f'Ratio:{ratio}')
        print()

        if torch.cuda.is_available():
            device = torch.device('cuda:0')
            print('running on GPU')
            # if you are using GPU
            torch.cuda.manual_seed(manualSeed)
            torch.cuda.manual_seed_all(manualSeed)

            torch.backends.cudnn.enabled = False
            torch.backends.cudnn.benchmark = False
            torch.backends.cudnn.deterministic = False
            os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' # !!! necessary for the below line
            torch.use_deterministic_algorithms(True)
            print('TRAIN deterministic algorithms')

        else:
            device = torch.device("cpu")
            print('running on CPU')

        # change the save path
        args.save_path = args.save_path+f'{args.save_name}/'

        # Choosing the right get_order function
        if args.mode == 'weighted_weight_exchange':
            get_order =  get_order_weighted
        else:
            get_order =  get_order_gnns


        # Getting our fold dict
        fold_dict,X = mf.create_fold_dict_new(dataset,num_hospitals=4,num_folds=5)

        # Perform the 5-fold Cross-Validation
        num_hospitals = args.num_folds - 1
        for f in range(args.num_folds):

            # fix the seeds
            np.random.seed(manualSeed)
            random.seed(manualSeed)
            torch.manual_seed(manualSeed)

            tic0 = timeit.default_timer()

            print(
                f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')

                        # Create hospitals
            hospitals = []
            for i in range(num_hospitals):
                  hospitals.append(Hospital_gnns(args,device))


            # Train data for each hospital
            train_data_list = []

            # Current test data
            test_data = X[-1][f].to(device)

            for i in range(num_hospitals):

                # Append the fold related to Hospital i - fold f. Note, here we append the real data, not the indices
                train_data_list.append(X[i][fold_dict[f'Hospital_{i}'][f'fold_{f}'][0]].to(device))


            # Start measuring the epochs time
            epochs_start = time.time()

            # Initiate Training
            for epoch in range(args.num_epochs):

                  epoch +=1

                  # order the hospitals based on the data availability
                  ordered_hospitals = get_order_gnns(table)

                  for h_i,hospital in enumerate(hospitals):

                    # get the train data for the hospital
                    train_data = train_data_list[h_i]

                    # get the table for th current hospital
                    table_hospital = table[h_i]

                    # Train the current hospital at the current timepoint for 1 epoch
                    mae_loss,hospital = train_one_epoch_gnns(args,hospital,train_data,table_hospital)

                    # Updating the hospital
                    hospitals[h_i] = hospital

                    if verbose:
                        print(f'Epoch:{epoch} Hospital:{h_i}, Train MAE Loss:{mae_loss}')


                  # Perform validation during training
                  if train_validate_verbose and (epoch%train_validate_verbosity_epochs==0 or epoch==1):

                      val_loss,val_mean_loss = validate_during_training_gnns(args, hospitals, test_data)
                      print(f"Epoch:{epoch},val_loss:")
                      print(f'Total MAE Loss')
                      print(val_loss)
                      print()
                      print(f'Average MAE Loss:')
                      print(val_mean_loss)
                      print()

                      for h_i,l in enumerate(val_mean_loss):

                          hospitals[h_i].scheduler.step(l)


                  if epoch != args.num_epochs - 1 or epoch != 0:
                      if epoch % args.C == 0 and args.mode != "4D-GNN":
                          print('Central Aggregation')
                          hospitals = update_main_by_average_gnns(hospitals)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN+":
                          hospitals = exchange_models(hospitals, t)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN++":
                          print('4D-FED-GNN++')
                          hospitals = exchange_models_based_on_order_gnns(hospitals, ordered_hospitals)
                      elif epoch % args.D == 0 and args.mode == "weighted_weight_exchange":
                          print('weighted_weight_exchange')
                          hospitals = exchange_models_weights_pairs_extreme(hospitals, t, ordered_hospitals)



            epochs_end = time.time()-epochs_start
            print()
            print(f'epochs finished with time:{epochs_end}')
            print()
            validate_gnns(args, hospitals, test_data, f)
            tic1 = timeit.default_timer()
            timer(tic0,tic1)



### CNN Simple

In [ ]:
import torch
from torch import nn

class ConvolutionalNN_1(nn.Module):
    def __init__(self):
        super(ConvolutionalNN_1, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Flatten()  # Flatten the 2D outputs from the conv layer
        )

        self.fc = nn.Linear(128*17*17, 35*35)  # 35*35, the size of output image

    def forward(self, data):

        data = data.view(1,1,data.shape[0],data.shape[1])
        data = self.conv(data)

        out = F.relu(self.fc(data))
        return out.view(35, 35)  # Reshape back into image format

In [ ]:
class ConvolutionalNN_1(nn.Module):
    def __init__(self):
        super(ConvolutionalNN_1, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Flatten()  # Flatten the 2D outputs from the conv layer
        )

        self.fc = nn.Linear(128*17*17, 35*35)  # 35*35, the size of output image

    def forward(self, data):

        data = data.view(1,1,data.shape[0],data.shape[1])
        data = self.conv(data)

        out = F.relu(self.fc(data))
        return out.view(35, 35)  # Reshape back into image format

In [ ]:
model = ConvolutionalNN_1()
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total parameters:", total_params)
print("Trainable parameters:", trainable_params)

Total parameters: 45390921
Trainable parameters: 45390921


In [ ]:
class Hospital_cnns_1():
    def __init__(self, args,device):
        """
        Hospital object contains a GNN and an optimizer for each timepoint

        Hospital object can update GNN-layer wise weights of its GNNs
        """

        self.model = ConvolutionalNN_1().to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min',factor=0.5,patience=3,verbose=True)

In [ ]:
import torch.nn.functional as F

class ConvolutionalNN_2(nn.Module):
    def __init__(self):
        super(ConvolutionalNN_2, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(1024, 2048, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),


            nn.MaxPool2d(2,2),

            nn.Flatten()  # Flatten the 2D outputs from the conv layer
        )

        # Calculate the size of the flattened layer
        flattened_size = 2048 * 4 * 4  # update the size based on the conv layers
        self.fc = nn.Linear(flattened_size, 35*35)  # 35*35, the size of output image

    def forward(self, data):
        data = data.view(1,1,data.shape[0],data.shape[1])
        data = self.conv(data)
        out = F.relu(self.fc(data))
        return out.view(35, 35)  # Reshape back into image format


In [ ]:
class Hospital_cnns_2():
    def __init__(self, args,device):
        """
        Hospital object contains a GNN and an optimizer for each timepoint

        Hospital object can update GNN-layer wise weights of its GNNs
        """

        self.model = ConvolutionalNN_2().to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min',factor=0.5,patience=3,verbose=True)

In [ ]:
import torch.nn.functional as F

class ConvolutionalNN_2_dropout(nn.Module):
    def __init__(self):
        super(ConvolutionalNN_2_dropout, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.2),  # Dropout after first layer
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.2),  # Dropout after second layer
            nn.MaxPool2d(2,2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),  # Increase dropout rate as network goes deeper
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),  # Increase dropout rate as network goes deeper
            nn.MaxPool2d(2,2),

            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.4),  # Increase dropout rate as network goes deeper
            nn.Conv2d(1024, 2048, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.4),  # Increase dropout rate as network goes deeper
            nn.MaxPool2d(2,2),

            nn.Flatten()  # Flatten the 2D outputs from the conv layer
        )

        # Calculate the size of the flattened layer
        flattened_size = 2048 * 4 * 4  # update the size based on the conv layers
        self.fc = nn.Linear(flattened_size, 35*35)  # 35*35, the size of output image

    def forward(self, data):
        data = data.view(1,1,data.shape[0],data.shape[1])
        data = self.conv(data)
        out = F.relu(self.fc(data))
        return out.view(35, 35)  # Reshape back into image format



In [ ]:
class Hospital_cnns_2_dropout():
    def __init__(self, args,device):
        """
        Hospital object contains a GNN and an optimizer for each timepoint

        Hospital object can update GNN-layer wise weights of its GNNs
        """

        self.model = ConvolutionalNN_2_dropout().to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min',factor=0.5,patience=3,verbose=True)

In [ ]:
model = ConvolutionalNN_2_dropout()
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total parameters:", total_params)
print("Trainable parameters:", trainable_params)



Total parameters: 65287881
Trainable parameters: 65287881


In [ ]:
import torch.nn.functional as F

import torch.nn as nn
import torch.nn.functional as F

class ConvolutionalNN_3(nn.Module):
    def __init__(self):
        super(ConvolutionalNN_3, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.2),

            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),

            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),

            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.4),

            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Flatten()  # Flatten the 2D outputs from the conv layer
        )

        flattened_size = 512 * 2 * 2  # update the size based on the conv layers
        self.fc = nn.Linear(flattened_size, 35*35)  # 35*35, the size of output image

    def forward(self, data):
        data = data.view(1,1,data.shape[0],data.shape[1])
        data = self.conv(data)
        out = F.relu(self.fc(data))
        return out.view(35, 35)  # Reshape back into image format


In [ ]:
model(dataset[0][0]).shape

torch.Size([35, 35])

In [ ]:
model = ConvolutionalNN_3()
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total parameters:", total_params)
print("Trainable parameters:", trainable_params)

Total parameters: 11895113
Trainable parameters: 11895113


In [ ]:
class Hospital_cnns_3():
    def __init__(self, args,device):
        """
        Hospital object contains a GNN and an optimizer for each timepoint

        Hospital object can update GNN-layer wise weights of its GNNs
        """

        self.model = ConvolutionalNN_3().to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min',factor=0.5,patience=3,verbose=True)

In [ ]:
def train_cnns(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1):
        """
            Arguments:
            args: arguments
            dataset: the whole dataset (train and test set)
            table: [num_hospitals, num_timepoints], holds timepoint-wise availability of hospitals

        This function performs training and testing reporting Mean Absolute Error (MAE) of the testing brain graphs.
        """

        # Create the results folders
        print('Train NEW')
        os.makedirs(args.save_path, exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/real_and_predicted_graphs', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/tp_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/total_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/test_mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/trained_models', exist_ok=True)


        # Create the results folders
        manualSeed = seed

        np.random.seed(manualSeed)
        random.seed(manualSeed)
        torch.manual_seed(manualSeed)

        # show the fixed seed
        print(f'Fixed seed:{seed}')
        print()

        # create the table that shows the data availability by  timepoint
        table = np.zeros((args.num_folds - 1, args.num_timepoints))
        table = random_table(args, ratio)
        print(f'Table:')
        print(table)
        print(f'Ratio:{ratio}')
        print()

        if torch.cuda.is_available():
            device = torch.device('cuda:0')
            print('running on GPU')
            # if you are using GPU
            torch.cuda.manual_seed(manualSeed)
            torch.cuda.manual_seed_all(manualSeed)

            torch.backends.cudnn.enabled = False
            torch.backends.cudnn.benchmark = False
            torch.backends.cudnn.deterministic = True
            os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' # !!! necessary for the below line
            torch.use_deterministic_algorithms(False)
            print('TRAIN deterministic algorithms')

        else:
            device = torch.device("cpu")
            print('running on CPU')

        # change the save path
        args.save_path = args.save_path+f'{args.save_name}/'

        # Choosing the right get_order function
        if args.mode == 'weighted_weight_exchange':
            get_order =  get_order_weighted
        else:
            get_order =  get_order_gnns


        # Getting our fold dict
        fold_dict,X = mf.create_fold_dict_new(dataset,num_hospitals=4,num_folds=5)

        # Perform the 5-fold Cross-Validation
        num_hospitals = args.num_folds - 1
        for f in range(args.num_folds):

            # fix the seeds
            np.random.seed(manualSeed)
            random.seed(manualSeed)
            torch.manual_seed(manualSeed)

            tic0 = timeit.default_timer()

            print(
                f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')

                        # Create hospitals
            hospitals = []
            for i in range(num_hospitals):
                  hospitals.append(Hospital_vgg(args,device))
                  print('VGG')


            # Train data for each hospital
            train_data_list = []

            # Current test data
            test_data = X[-1][f].to(device)

            for i in range(num_hospitals):

                # Append the fold related to Hospital i - fold f. Note, here we append the real data, not the indices
                train_data_list.append(X[i][fold_dict[f'Hospital_{i}'][f'fold_{f}'][0]].to(device))


            # Start measuring the epochs time
            epochs_start = time.time()

            # Initiate Training
            for epoch in range(args.num_epochs):

                  epoch +=1

                  # order the hospitals based on the data availability
                  ordered_hospitals = get_order_gnns(table)

                  for h_i,hospital in enumerate(hospitals):

                    # get the train data for the hospital
                    train_data = train_data_list[h_i]

                    # get the table for th current hospital
                    table_hospital = table[h_i]

                    # Train the current hospital at the current timepoint for 1 epoch
                    mae_loss,hospital = train_one_epoch_gnns(args,hospital,train_data,table_hospital)

                    # Updating the hospital
                    hospitals[h_i] = hospital

                    if verbose:
                        print(f'Epoch:{epoch} Hospital:{h_i}, Train MAE Loss:{mae_loss}')


                  # Perform validation during training
                  if train_validate_verbose and (epoch%train_validate_verbosity_epochs==0 or epoch==1):

                      val_loss,val_mean_loss = validate_during_training_gnns(args, hospitals, test_data)
                      print(f"Epoch:{epoch},val_loss:")
                      print(f'Total MAE Loss')
                      print(val_loss)
                      print()
                      print(f'Average MAE Loss:')
                      print(val_mean_loss)
                      print()

                      for h_i,l in enumerate(val_mean_loss):

                          hospitals[h_i].scheduler.step(l)


                  if epoch != args.num_epochs - 1 or epoch != 0:
                      if epoch % args.C == 0 and args.mode != "4D-GNN":
                          print('Central Aggregation')
                          hospitals = update_main_by_average_gnns(hospitals)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN+":
                          hospitals = exchange_models(hospitals, t)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN++":
                          print('4D-FED-GNN++')
                          hospitals = exchange_models_based_on_order_gnns(hospitals, ordered_hospitals)
                      elif epoch % args.D == 0 and args.mode == "weighted_weight_exchange":
                          print('weighted_weight_exchange')
                          hospitals = exchange_models_weights_pairs_extreme(hospitals, t, ordered_hospitals)



            epochs_end = time.time()-epochs_start
            print()
            print(f'epochs finished with time:{epochs_end}')
            print()
            validate_gnns(args, hospitals, test_data, f)
            tic1 = timeit.default_timer()
            timer(tic0,tic1)



In [ ]:
def train_cnns_custom(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1):
        """
            Arguments:
            args: arguments
            dataset: the whole dataset (train and test set)
            table: [num_hospitals, num_timepoints], holds timepoint-wise availability of hospitals

        This function performs training and testing reporting Mean Absolute Error (MAE) of the testing brain graphs.
        """

        # Create the results folders
        print('Train NEW')
        os.makedirs(args.save_path, exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/real_and_predicted_graphs', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/tp_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/train_losses/total_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/test_mae_losses', exist_ok=True)
        os.makedirs(args.save_path+f'{args.save_name}/trained_models', exist_ok=True)


        # Create the results folders
        manualSeed = seed

        np.random.seed(manualSeed)
        random.seed(manualSeed)
        torch.manual_seed(manualSeed)

        # show the fixed seed
        print(f'Fixed seed:{seed}')
        print()

        # create the table that shows the data availability by  timepoint
        table = np.zeros((args.num_folds - 1, args.num_timepoints))
        table = random_table(args, ratio)
        print(f'Table:')
        print(table)
        print(f'Ratio:{ratio}')
        print()

        if torch.cuda.is_available():
            device = torch.device('cuda:0')
            print('running on GPU')
            # if you are using GPU
            torch.cuda.manual_seed(manualSeed)
            torch.cuda.manual_seed_all(manualSeed)

            torch.backends.cudnn.enabled = False
            torch.backends.cudnn.benchmark = False
            torch.backends.cudnn.deterministic = True
            os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' # !!! necessary for the below line
            torch.use_deterministic_algorithms(False)
            print('TRAIN deterministic algorithms')

        else:
            device = torch.device("cpu")
            print('running on CPU')

        # change the save path
        args.save_path = args.save_path+f'{args.save_name}/'

        # Choosing the right get_order function
        if args.mode == 'weighted_weight_exchange':
            get_order =  get_order_weighted
        else:
            get_order =  get_order_gnns


        # Getting our fold dict
        fold_dict,X = mf.create_fold_dict_new(dataset,num_hospitals=4,num_folds=5)

        # Perform the 5-fold Cross-Validation
        num_hospitals = args.num_folds - 1
        for f in range(args.num_folds):

            # fix the seeds
            np.random.seed(manualSeed)
            random.seed(manualSeed)
            torch.manual_seed(manualSeed)

            tic0 = timeit.default_timer()

            print(
                f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')

                        # Create hospitals
            hospitals = []
            for i in range(num_hospitals):
                  hospitals.append(Hospital_vgg(args,device))


            # Train data for each hospital
            train_data_list = []

            # Current test data
            test_data = X[-1][f].to(device)

            for i in range(num_hospitals):

                # Append the fold related to Hospital i - fold f. Note, here we append the real data, not the indices
                train_data_list.append(X[i][fold_dict[f'Hospital_{i}'][f'fold_{f}'][0]].to(device))


            # Start measuring the epochs time
            epochs_start = time.time()

            # Initiate Training
            for epoch in range(args.num_epochs):

                  epoch +=1

                  # order the hospitals based on the data availability
                  ordered_hospitals = get_order_gnns(table)

                  for h_i,hospital in enumerate(hospitals):

                    # get the train data for the hospital
                    train_data = train_data_list[h_i]

                    # get the table for th current hospital
                    table_hospital = table[h_i]

                    # Train the current hospital at the current timepoint for 1 epoch
                    mae_loss,hospital = train_one_epoch_gnns(args,hospital,train_data,table_hospital)

                    # Updating the hospital
                    hospitals[h_i] = hospital

                    if verbose:
                        print(f'Epoch:{epoch} Hospital:{h_i}, Train MAE Loss:{mae_loss}')


                  # Perform validation during training
                  if train_validate_verbose and (epoch%train_validate_verbosity_epochs==0 or epoch==1):

                      val_loss,val_mean_loss = validate_during_training_gnns(args, hospitals, test_data)
                      print(f"Epoch:{epoch},val_loss:")
                      print(f'Total MAE Loss')
                      print(val_loss)
                      print()
                      print(f'Average MAE Loss:')
                      print(val_mean_loss)
                      print()

                      for h_i,l in enumerate(val_mean_loss):

                          hospitals[h_i].scheduler.step(l)


                  if epoch != args.num_epochs - 1 or epoch != 0:
                      if epoch % args.C == 0 and args.mode != "4D-GNN":
                          print('Central Aggregation')
                          hospitals = update_main_by_average_gnns_new(hospitals,ordered_hospitals,table)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN+":
                          hospitals = exchange_models(hospitals, t)
                      elif epoch % args.D == 0 and args.mode == "4D-FED-GNN++":
                          print('4D-FED-GNN++')
                          hospitals = exchange_models_based_on_order_gnns(hospitals,ordered_hospitals)
                      elif epoch % args.D == 0 and args.mode == "weighted_weight_exchange":
                          print('weighted_weight_exchange')
                          hospitals = exchange_models_weights_pairs_extreme(hospitals, t, ordered_hospitals)



            epochs_end = time.time()-epochs_start
            print()
            print(f'epochs finished with time:{epochs_end}')
            print()
            validate_gnns(args, hospitals, test_data, f)
            tic1 = timeit.default_timer()
            timer(tic0,tic1)



In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

#### CNN 1

In [ ]:
# table = 4/8 CNN1
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='Simple_CNN_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=0/4,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 0. 1.]
 [1. 0. 1.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.0

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------


NameError: ignored

In [ ]:
torch.cuda.empty_cache()


#### CNN 2

In [ ]:
# seed = 10 table = 4/8 fixed folds CNN_2 dropout
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='CNN2_dropout_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Epoch:1,val_loss:
Total MAE Loss
[[0.07516651 0.07205517]
 [0.0858584  0.08280423]
 [0.06787438 0.06496757]
 [0.07490906 0.07166309]]

Average MAE Loss:
[0.07361084 0.08433132 0.06642098 0.07328607]

Epoch:2,val_loss:
Total MAE Loss
[[0.07498428 0.07173545]
 [0.0858584  0.08280423]
 [0.06639742 0.0638234 ]
 [0.06206009 0.05967656]]

Average MAE Loss:
[0.07335987 0.08433132 0.06511041 0.06086833]

Epoch:3,val_loss:
Total MAE Loss
[[0.07476131 0.07152765]
 [0.0858584  0.08280423]
 [0.06700019 0.06495788]
 [0.05992341

In [ ]:
# seed = 10 table = 2/8 fixed folds CNN_2 dropout
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/2_8/global_test/GNNs/'
args.save_name='CNN2_dropout_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=2/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.25

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Epoch:1,val_loss:
Total MAE Loss
[[0.08581297 0.0827677 ]
 [0.0858584  0.08280423]
 [0.07732824 0.07402257]
 [0.05657713 0.05247714]]

Average MAE Loss:
[0.08429033 0.08433132 0.0756754  0.05452714]

Epoch:2,val_loss:
Total MAE Loss
[[0.08581297 0.0827677 ]
 [0.0858584  0.08280423]
 [0.05827196 0.05617904]
 [0.06396585 0.06359772]]

Average MAE Loss:
[0.08429033 0.08433132 0.0572255  0.06378178]

Epoch:3,val_loss:
Total MAE Loss
[[0.08581297 0.0827677 ]
 [0.0858584  0.08280423]
 [0.0561692  0.05395243]
 [0.0576813

In [ ]:
# seed = 10 table = 4/8 fixed folds CNN_2 dropout
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='CNN2_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Epoch:1,val_loss:
Total MAE Loss
[[0.07953594 0.07624164]
 [0.0858584  0.08280423]
 [0.05980194 0.05644431]
 [0.05575079 0.0533058 ]]

Average MAE Loss:
[0.07788879 0.08433132 0.05812313 0.0545283 ]

Epoch:2,val_loss:
Total MAE Loss
[[0.07526286 0.07202254]
 [0.0858584  0.08280423]
 [0.051752   0.05004802]
 [0.05443482 0.05243137]]

Average MAE Loss:
[0.0736427  0.08433132 0.05090001 0.0534331 ]

Epoch:3,val_loss:
Total MAE Loss
[[0.07593101 0.07298071]
 [0.0858584  0.08280423]
 [0.05155187 0.04938125]
 [0.05168367

In [ ]:
# seed = 10 table = 2/8 fixed folds CNN2
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/2_8/global_test/GNNs/'
args.save_name='CNN2_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=2/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.25

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Epoch:1,val_loss:
Total MAE Loss
[[0.08581297 0.0827677 ]
 [0.0858584  0.08280423]
 [0.05980194 0.05644431]
 [0.05575079 0.0533058 ]]

Average MAE Loss:
[0.08429033 0.08433132 0.05812313 0.0545283 ]

Epoch:2,val_loss:
Total MAE Loss
[[0.08581297 0.0827677 ]
 [0.0858584  0.08280423]
 [0.051752   0.05004802]
 [0.05443482 0.05243137]]

Average MAE Loss:
[0.08429033 0.08433132 0.05090001 0.0534331 ]

Epoch:3,val_loss:
Total MAE Loss
[[0.08581297 0.0827677 ]
 [0.0858584  0.08280423]
 [0.05155187 0.04938125]
 [0.0516836

In [ ]:
# seed = 10 table = 4/8 fixed folds CNN_2 dropout
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='Try_CNN_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Epoch:1,val_loss:
Total MAE Loss
[[0.07632864 0.07342375]
 [0.0858584  0.08280423]
 [0.06958028 0.06679701]
 [0.0743964  0.07121451]]

Average MAE Loss:
[0.0748762  0.08433132 0.06818865 0.07280545]

Epoch:2,val_loss:
Total MAE Loss
[[0.07513348 0.07181109]
 [0.0858584  0.08280423]
 [0.07235302 0.06956607]
 [0.06359088 0.06163194]]

Average MAE Loss:
[0.07347229 0.08433132 0.07095955 0.06261141]

Epoch:3,val_loss:
Total MAE Loss
[[0.07448399 0.07126369]
 [0.0858584  0.08280423]
 [0.06782266 0.06516868]
 [0.0609222 

In [ ]:
# seed = 10 table = 4/8 fixed folds CNN_2 dropout
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='Try_Simple2_CNN_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Epoch:1,val_loss:
Total MAE Loss
[[0.07632864 0.07342375]
 [0.0858584  0.08280423]
 [0.06958028 0.06679701]
 [0.0743964  0.07121451]]

Average MAE Loss:
[0.0748762  0.08433132 0.06818865 0.07280545]

Epoch:2,val_loss:
Total MAE Loss
[[0.07513348 0.07181109]
 [0.0858584  0.08280423]
 [0.07235302 0.06956607]
 [0.06359088 0.06163194]]

Average MAE Loss:
[0.07347229 0.08433132 0.07095955 0.06261141]

Epoch:3,val_loss:
Total MAE Loss
[[0.07448399 0.07126369]
 [0.0858584  0.08280423]
 [0.06782266 0.06516868]
 [0.0609222 

In [ ]:
# seed = 10 table = 2/8 fixed folds CNN_2
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/2_8/global_test/GNNs/'
args.save_name='Simple2_CNN_Dropout_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=2/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.25

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Epoch:1,val_loss:
Total MAE Loss
[[0.08581297 0.0827677 ]
 [0.0858584  0.08280423]
 [0.06059251 0.05770964]
 [0.05660847 0.05246856]]

Average MAE Loss:
[0.08429034 0.08433132 0.05915108 0.05453852]

Epoch:2,val_loss:
Total MAE Loss
[[0.08581297 0.0827677 ]
 [0.0858584  0.08280423]
 [0.06812717 0.06469029]
 [0.06685961 0.06678156]]

Average MAE Loss:
[0.08429034 0.08433132 0.06640873 0.06682059]

Epoch:3,val_loss:
Total MAE Loss
[[0.08581297 0.0827677 ]
 [0.0858584  0.08280423]
 [0.06874192 0.06777871]
 [0.0603156

KeyboardInterrupt: ignored

<Figure size 640x480 with 0 Axes>

In [ ]:
# seed = 10 table = 4/8 fixed folds CNN_2
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='Simple2_CNN_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Epoch:1,val_loss:
Total MAE Loss
[[0.07423845 0.07099184]
 [0.0858584  0.08280423]
 [0.05980123 0.05644355]
 [0.05581843 0.05272923]]

Average MAE Loss:
[0.07261514 0.08433132 0.05812239 0.05427383]

Epoch:2,val_loss:
Total MAE Loss
[[0.05363391 0.05099773]
 [0.0858584  0.08280423]
 [0.05174453 0.05002043]
 [0.05459361 0.0530942 ]]

Average MAE Loss:
[0.05231582 0.08433132 0.05088248 0.05384391]

Epoch:3,val_loss:
Total MAE Loss
[[0.05071445 0.04856771]
 [0.0858584  0.08280423]
 [0.05152505 0.04936152]
 [0.05067437

In [ ]:
# seed = 10 table = 4/8 fixed folds CNN_2 dropout
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='Try_Simple2_CNN_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

In [ ]:
# seed = 500 table = 4/8 fixed folds CNN_2
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_500/4_8/global_test/GNNs/'
args.save_name='Simple2_CNN_4D-FED-GNN++'
train_cnns(args, dataset,seed=500,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:500

Table:
[[1. 0. 0.]
 [1. 0. 1.]
 [1. 1. 0.]
 [1. 1. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Epoch:1,val_loss:
Total MAE Loss
[[0.07742107 0.09364641]
 [0.07161249 0.08753487]
 [0.06609653 0.08078363]
 [0.05705494 0.06765399]]

Average MAE Loss:
[0.08553374 0.07957368 0.07344008 0.06235446]

Epoch:2,val_loss:
Total MAE Loss
[[0.07742107 0.09364641]
 [0.07070872 0.08634324]
 [0.06591281 0.08030224]
 [0.05279149 0.06364293]]

Average MAE Loss:
[0.08553374 0.07852598 0.07310752 0.05821721]

Epoch:3,val_loss:
Total MAE Loss
[[0.07742107 0.09364641]
 [0.07447436 0.08846008]
 [0.06601703 0.08011893]
 [0.0537879

In [ ]:
args.C=50
args.D=50

In [ ]:
# seed = 500 table = 4/8 fixed folds CNN_2
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_500/4_8/global_test/GNNs/'
args.save_name='Try'
train_cnns(args, dataset,seed=500,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:500

Table:
[[1. 0. 0.]
 [1. 0. 1.]
 [1. 1. 0.]
 [1. 1. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Epoch:1,val_loss:
Total MAE Loss
[[0.07742107 0.09364641]
 [0.07161249 0.08753487]
 [0.06609653 0.08078363]
 [0.05705494 0.06765399]]

Average MAE Loss:
[0.08553374 0.07957368 0.07344008 0.06235446]

Epoch:2,val_loss:
Total MAE Loss
[[0.07742107 0.09364641]
 [0.07070872 0.08634324]
 [0.06591281 0.08030224]
 [0.05279149 0.06364293]]

Average MAE Loss:
[0.08553374 0.07852598 0.07310752 0.05821721]

Epoch:3,val_loss:
Total MAE Loss
[[0.07742107 0.09364641]
 [0.07447436 0.08846008]
 [0.06601703 0.08011893]
 [0.0537879

KeyboardInterrupt: ignored

#### CNN3

In [ ]:
# seed = 10 table = 4/8 fixed folds CNN_2
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='Simple3_CNN_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------


KeyboardInterrupt: ignored

### Resnets

**Tried resnet 18,50,101 no good result**

In [ ]:
import torch.nn.functional as F
from torchvision import models

In [ ]:
class Resnet50(nn.Module):
    def __init__(self):
        super(Resnet50, self).__init__()

        # Load a pre-trained ResNet model and remove the last layer (fc layer)
        self.resnet = models.resnet18(weights=None)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # Modify the first layer to get 1 input channel
        in_features = self.resnet.fc.in_features

        # Replace the last layer (fc layer) with a new one
        self.resnet.fc = nn.Linear(in_features, 35*35)

    def forward(self, x):
        # Pass the input tensor through each of our operations (or 'layers')
        x = x.view(1,1,x.shape[0],x.shape[1])
        pred = F.relu(self.resnet(x))
        pred = pred.view(35,35)
        return pred


In [ ]:
model = Resnet50()
model(data)

tensor([[0.0000, 0.0000, 0.0951,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3166, 0.0000,  ..., 0.2617, 0.5820, 0.8956],
        [0.0000, 0.0000, 0.0742,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0309,  ..., 0.7270, 0.0693, 0.0000],
        [0.0000, 0.0000, 0.0337,  ..., 0.0000, 0.5709, 0.0000],
        [0.3863, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<ViewBackward0>)

In [ ]:
model = Resnet50()
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total parameters:", total_params)
print("Trainable parameters:", trainable_params)


Total parameters: 11798665
Trainable parameters: 11798665


In [ ]:
class Hospital_resnet50():
    def __init__(self, args,device):
        """
        Hospital object contains a GNN and an optimizer for each timepoint

        Hospital object can update GNN-layer wise weights of its GNNs
        """

        self.model = Resnet50().to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min',factor=0.5,patience=5,verbose=True)

In [ ]:
# seed = 10 table = 4/8 fixed folds CNN_2 dropout
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='Resnet50_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
Epoch:1,val_loss:
Total MAE Loss
[[0.08592845 0.08281599]
 [0.07618625 0.0732338 ]
 [0.08539023 0.08210879]
 [0.11087855 0.10611269]]

Average MAE Loss:
[0.08437222 0.07471003 0.08374951 0.10849562]

Epoch:2,val_loss:
Total MAE Loss
[[0.08661643 0.08300977]
 [0.08161425 0.07880727]
 [0.08653458 0.08286455]
 [0.08588463 0.08277113]]

Average MAE Loss:
[0.0848131  0.08021076 0.08469957 0.08432788]

Epoch:3,val_loss:
Total MAE Loss
[[0.08675836 0.08305413]
 [0.09307665 0.09023138]
 [0.08681499 0.08309603]
 [0.0858959 

KeyboardInterrupt: ignored

### VGG

In [ ]:
import torch.nn.functional as F
from torchvision import models

In [ ]:
models.vgg19(weights=None)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [ ]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()

        # Load a VGG model and remove the last layer (fc layer)
        self.vgg = models.vgg19(weights=None)  # Turn off pre-training
        self.vgg.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)  # Modify the first layer to get 1 input channel
        in_features = self.vgg.classifier[6].in_features

        # Replace the last layer (fc layer) with a new one
        self.vgg.classifier[6] = nn.Linear(in_features, 35*35)

    def forward(self, x):
        # Pass the input tensor through each of our operations (or 'layers')
        x = x.view(1,1,x.shape[0],x.shape[1])
        pred = F.relu(self.vgg(x))
        pred = pred.view(35,35)
        return pred


In [ ]:
model = VGG()
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total parameters:", total_params)
print("Trainable parameters:", trainable_params)

Total parameters: 144587913
Trainable parameters: 144587913


In [ ]:
class Hospital_vgg():
    def __init__(self, args,device):
        """
        Hospital object contains a GNN and an optimizer for each timepoint

        Hospital object can update GNN-layer wise weights of its GNNs
        """

        self.model = VGG().to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min',factor=0.5,patience=5,verbose=True)

In [ ]:
# seed = 10 table = 4/8 fixed folds VGG19
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/4_8/global_test/GNNs/'
args.save_name='VGG19_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=4/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.5

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
VGG
VGG
VGG
VGG
Epoch:1,val_loss:
Total MAE Loss
[[0.05137331 0.04872084]
 [0.08400421 0.08107341]
 [0.05294302 0.0504333 ]
 [0.05192866 0.04903244]]

Average MAE Loss:
[0.05004708 0.08253881 0.05168816 0.05048055]

Epoch:2,val_loss:
Total MAE Loss
[[0.05631251 0.05447202]
 [0.08400421 0.08107341]
 [0.04722659 0.04550089]
 [0.04832608 0.0464755 ]]

Average MAE Loss:
[0.05539227 0.08253881 0.04636374 0.04740079]

Epoch:3,val_loss:
Total MAE Loss
[[0.04171986 0.03899236]
 [0.08400421 0.08107341]
 [0.04533347 0.043324

In [ ]:
# seed = 10 table = 2/8 fixed folds VGG19
args.mode='4D-FED-GNN++'
args.save_path = '/content/drive/MyDrive/Dissertation/4D-FedGNN-Plus_mine/Real/manual_seed_10/2_8/global_test/GNNs/'
args.save_name='VGG19_4D-FED-GNN++'
train_cnns(args, dataset,seed=10,ratio=2/8,verbose=False,train_validate_verbose=True,train_validate_verbosity_epochs=1)

Train NEW
Fixed seed:10

Table:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
Ratio:0.25

running on GPU
TRAIN deterministic algorithms

Creating the fold dictionary...

Data for hospital 0
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 1
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 2
fold:0
fold:1
fold:2
fold:3
fold:4
Data for hospital 3
fold:0
fold:1
fold:2
fold:3
fold:4
------------------------------------Fold [1/5]-----------------------------------------
VGG
VGG
VGG
VGG
Epoch:1,val_loss:
Total MAE Loss
[[0.08390902 0.08091476]
 [0.08400421 0.08107341]
 [0.06247937 0.05982874]
 [0.05456273 0.05181147]]

Average MAE Loss:
[0.08241189 0.08253881 0.06115406 0.0531871 ]

Epoch:2,val_loss:
Total MAE Loss
[[0.08390902 0.08091476]
 [0.08400421 0.08107341]
 [0.04634947 0.04490671]
 [0.05022066 0.04880004]]

Average MAE Loss:
[0.08241189 0.08253881 0.04562809 0.04951035]

Epoch:3,val_loss:
Total MAE Loss
[[0.08390902 0.08091476]
 [0.08400421 0.08107341]
 [0.04537708 0.04344

# Check


## seed 10 4_8

In [ ]:
# dropout 0.2->0.3->0.4
check(name='CNN2_dropout_4D-FED-GNN++',seed=10,ratio='4_8')

Hospital 0:
[0.04698905 0.0494738 ]
Hospital 1:
[0.04699336 0.04948168]
Hospital 2:
[0.04695561 0.04947038]
Hospital 3:
[0.04703576 0.04953545]

Total sum:[0.04699344 0.04949033]


In [ ]:
# CNN2
check(name='CNN2_4D-FED-GNN++',seed=10,ratio='4_8')

Hospital 0:
[0.04868297 0.05111571]
Hospital 1:
[0.0486493  0.05113705]
Hospital 2:
[0.04863589 0.0511804 ]
Hospital 3:
[0.04865658 0.05114117]

Total sum:[0.04865619 0.05114359]


In [ ]:
#VGG19
check(name='VGG19_4D-FED-GNN++',seed=10,ratio='4_8')

Hospital 0:
[0.0450016  0.04764346]
Hospital 1:
[0.04515179 0.04779888]
Hospital 2:
[0.04527794 0.04818096]
Hospital 3:
[0.04554866 0.04799936]

Total sum:[0.045245   0.04790566]


In [ ]:
check(name='Simple2_CNN_4D-FED-GNN++',seed=10,ratio='4_8')

Hospital 0:
[0.04830841 0.05091142]
Hospital 1:
[0.04834165 0.05092986]
Hospital 2:
[0.04836772 0.05100498]
Hospital 3:
[0.04834954 0.05092387]

Total sum:[0.04834183 0.05094253]


## seed 10 2_8

In [ ]:
# dropout 0.2->0.3->0.4
check(name='CNN2_dropout_4D-FED-GNN++',seed=10,ratio='2_8')

Hospital 0:
[0.04888912 0.05139125]
Hospital 1:
[0.04888912 0.05139125]
Hospital 2:
[0.04854155 0.05121583]
Hospital 3:
[0.04854986 0.05125947]

Total sum:[0.04871741 0.05131445]


In [ ]:
# CNN2
check(name='CNN2_4D-FED-GNN++',seed=10,ratio='2_8')

Hospital 0:
[0.04960811 0.05220952]
Hospital 1:
[0.04960811 0.05220952]
Hospital 2:
[0.04949756 0.05219058]
Hospital 3:
[0.0494956  0.05217005]

Total sum:[0.04955234 0.05219492]


In [ ]:
#VGG16
check(name='VGG19_4D-FED-GNN++',seed=10,ratio='4_8')

Hospital 0:
[0.0450016  0.04764346]
Hospital 1:
[0.04515179 0.04779888]
Hospital 2:
[0.04527794 0.04818096]
Hospital 3:
[0.04554866 0.04799936]

Total sum:[0.045245   0.04790566]


In [ ]:

check(name='New_Custom_Simple2_CNN_4D-FED-GNN++',seed=10,ratio='4_8')

Hospital 0:
[0.04864759 0.05125555]
Hospital 1:
[0.04873149 0.0512792 ]
Hospital 2:
[0.04871789 0.05137924]
Hospital 3:
[0.04876227 0.05130414]

Total sum:[0.04871481 0.05130454]


In [ ]:
check(name='New_Custom_Simple2_CNN_4D-FED-GNN++',seed=10,ratio='4_8')

Hospital 0:
[0.04969393 0.05235872]
Hospital 1:
[0.0496548  0.05241038]
Hospital 2:
[0.04975457 0.05259932]
Hospital 3:
[0.04968568 0.05241673]

Total sum:[0.04969725 0.05244629]


In [ ]:
check(name='Simple2_CNN_4D-FED-GNN++',seed=10,ratio='4_8')

Hospital 0:
[0.04830841 0.05091142]
Hospital 1:
[0.04834165 0.05092986]
Hospital 2:
[0.04836772 0.05100498]
Hospital 3:
[0.04834954 0.05092387]

Total sum:[0.04834183 0.05094253]


## seed 500 4_8

In [ ]:
check(name='Simple2_CNN_4D-FED-GNN++',seed=500,ratio='4_8')

Hospital 0:
[0.03884535 0.04585896]
Hospital 1:
[0.03884614 0.04587628]
Hospital 2:
[0.03899311 0.04584705]
Hospital 3:
[0.03874649 0.04584995]

Total sum:[0.03885777 0.04585806]
